# Setup

For this project we have chosen to use google colab to run our code. The reason is that free gpu is provided which we can use in the later part of this report to speed up our GNN learning process. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In order for this notebook to work you have to commit to the following steps : 

1. Download the project folder from our [github page](https://github.com/Potamitisn/NML_Project) and upload it to your Google Drive.

2. In the following code block change the `project_path` to the path leading where you saved you the project folder.

3. You're good to go !

In [ ]:
project_path = '/content/drive/MyDrive/Colab Notebooks/network_machine_learning/project'
! cd project_path

import os
os.chdir(project_path)
current_path = os.getcwd()

figures_path = os.path.join(current_path, 'figures')
images_path = os.path.join(current_path, 'images')
gexf_path = os.path.join(current_path, 'gexf_files')
graphs_path = os.path.join(current_path, 'graphs')
features_path = os.path.join(current_path, 'features')

if not os.path.exists(gexf_path):
    os.makedirs(gexf_path)

if not os.path.exists(graphs_path):
    os.makedirs(graphs_path)

if not os.path.exists(features_path):
    os.makedirs(features_path)

if not os.path.exists(figures_path):
    os.makedirs(figures_path)

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

!pip install torchmetrics

from IPython.display import clear_output 
clear_output()

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
import scipy
import networkx as nx

%matplotlib inline

In [ ]:
from typing import Callable, List, Optional
import torch
import torchmetrics
import torch_geometric as pyg
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import GNNBenchmarkDataset, Planetoid
from torch_geometric.utils import from_networkx, to_networkx, get_laplacian
from torch_geometric.nn.conv import MessagePassing
from tqdm.notebook import tqdm
from torch_geometric.utils.convert import from_networkx

import matplotlib.pyplot as plt

In [ ]:
!pip install stellargraph
from gensim.models import Word2Vec
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
clear_output()

In [ ]:
from sklearn.model_selection import train_test_split
# additional imports are necessary
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans

In [ ]:
from plotly.subplots import make_subplots
from PIL import Image
import plotly.express as px

## Datasets

### Importing and displaying the datasets

In the beginning we will import our data into a `pandas.DataFrame`structure.

In [ ]:
DC_data_2000 = pd.read_parquet('data/dc_2000-2001.parquet', engine='pyarrow')
DC_data_2001 = pd.read_parquet('data/dc_2001-2002.parquet', engine='pyarrow')

marvel_data_2000 = pd.read_parquet('data/marvel_2000-2001.parquet', engine='pyarrow')
marvel_data_2001 = pd.read_parquet('data/marvel_2001-2002.parquet', engine='pyarrow')

The next set will be to sort our dataframe in ascending alphabetical order based on the `URL`column, reset `the index` and create a new column named `Id` whose values are copied from the `index` column. Finally, we will display a portion of the dataset to get a glimpse.

- We will repeat this process for each of the 4 datasets.

**DC 2000**

In [ ]:
DC_data_2000 = DC_data_2000.sort_values(by='URL').reset_index()
DC_data_2000 = DC_data_2000.reset_index().rename(columns={"index": "Id"})
DC_data_2000

**DC 2001**

In [ ]:
DC_data_2001 = DC_data_2001.sort_values(by='URL').reset_index()
DC_data_2001 = DC_data_2001.reset_index().rename(columns={"index": "Id"})
DC_data_2001

**Marvel 2000**

In [ ]:
marvel_data_2000 = marvel_data_2000.sort_values(by='URL').reset_index()
marvel_data_2000 = marvel_data_2000.reset_index().rename(columns={"index": "Id"})
marvel_data_2000

**Marvel 2001**

In [ ]:
marvel_data_2001 = marvel_data_2001.sort_values(by='URL').reset_index()
marvel_data_2001 = marvel_data_2001.reset_index().rename(columns={"index": "Id"})
marvel_data_2001

## Feature Engineering

In order to get a better understanding of our datasets we will use the `.info()`function to check the type of the values for each column and the number of null/non-null values.

In [ ]:
DC_data_2000.info()

In [ ]:
DC_data_2001.info()

In [ ]:
marvel_data_2000.info()

In [ ]:
marvel_data_2001.info()

From above printing, we can see there is no missing value for features in the datasets. However, when we displayed the datasets earlier we have noticed that in the **Relatives** and **Affiliation** features/columns there were instead some empty lists of the form `[]`. In the following, we will take a look into each dataset and check the number of times there is an empty list in either of these features or even **Comics** too.

In [ ]:
empty_relatives_marvel_2000 = marvel_data_2000[marvel_data_2000['Relatives'].apply(lambda x: len(x)==0)]
empty_affliation_marvel_2000 = marvel_data_2000[marvel_data_2000['Affiliation'].apply(lambda x: len(x)==0)]
empty_comics_marvel_2000 = marvel_data_2000[marvel_data_2000['Comics'].apply(lambda x: len(x)==0)]
print('For marvel 2000 dataset, there are {} empty lists in Relatives feature, {} empty lists in Affliation feature and {} empty lists in Comics feature.'.format(len(empty_relatives_marvel_2000), len(empty_affliation_marvel_2000), len(empty_comics_marvel_2000)))

In [ ]:
empty_relatives_marvel_2001 = marvel_data_2001[marvel_data_2001['Relatives'].apply(lambda x: len(x)==0)]
empty_affliation_marvel_2001 = marvel_data_2001[marvel_data_2001['Affiliation'].apply(lambda x: len(x)==0)]
empty_comics_marvel_2001 = marvel_data_2001[marvel_data_2001['Comics'].apply(lambda x: len(x)==0)]
print('For marvel 2001 dataset, there are {} empty lists in Relatives feature, {} empty lists in Affliation feature and {} empty lists in Comics feature.'.format(len(empty_relatives_marvel_2001), len(empty_affliation_marvel_2001), len(empty_comics_marvel_2001)))

In [ ]:
empty_relatives_DC_2000 = DC_data_2000[DC_data_2000['Relatives'].apply(lambda x: len(x)==0)]
empty_affliation_DC_2000 = DC_data_2000[DC_data_2000['Affiliation'].apply(lambda x: len(x)==0)]
empty_comics_DC_2000 = DC_data_2000[DC_data_2000['Comics'].apply(lambda x: len(x)==0)]
print('For DC 2000 dataset, there are {} empty lists in Relatives feature, {} empty lists in Affliation feature and {} empty lists in Comics feature.'.format(len(empty_relatives_DC_2000), len(empty_affliation_DC_2000), len(empty_comics_DC_2000)))

In [ ]:
empty_relatives_DC_2001 = DC_data_2001[DC_data_2001['Relatives'].apply(lambda x: len(x)==0)]
empty_affliation_DC_2001 = DC_data_2001[DC_data_2001['Affiliation'].apply(lambda x: len(x)==0)]
empty_comics_DC_2001 = DC_data_2001[DC_data_2001['Comics'].apply(lambda x: len(x)==0)]
print('For DC 2001 dataset, there are {} empty lists in Relatives feature, {} empty lists in Affliation feature and {} empty lists in Comics feature.'.format(len(empty_relatives_DC_2001), len(empty_affliation_DC_2001), len(empty_comics_DC_2001)))

From above printings, we can see not all characters in Marvel and DC universe are involved in an Affiliation or have their own Relatives. However, all characters appeared in at least one Comic. Therefore, if we create the graph based on Affiliations and Relatives, not all characters have edges. If we create the graph based on Comics, all characters will be connected.


# Network creation

In this section, we consider create network based on **Affiliations**, **Relatives** and **Comics**. Considering above findings, we will normalize all the adjacency matrics to the same range of edge weight, and give small weight to Combics adjacency matrix.

## Utility Functions

To do this in the next block we will define some utility functions that will help us in this process.

In [ ]:
def get_edge_dataframe(df, edge_name):
    """
    Inputs
      df [pandas.DataFrame]: Initial dataframe
      edge_name [str]: Name of the column of the df that we want to use as edges to
                       create our network. For example Affiliations or Relatives.
    Outputs
      edge_df [pandas.DataFrame]: This dataframe will connect each pair of characters
                                  that share the same edge_name. Each row will portray
        the ids and the URLs of each character along with 1 edge_name eg. Affiliation, 
        which they share. Additionaly, a new column will be added called weight. It 
        will be of type = "int" and its value will be equal to the amount of affiliations 
        shared between the 2 characters. Output will have the following form : 
                  
                    Id_x|Id_y|weight|URL_x|edge_name|URL_y

    """
    df_source = df[['Id', 'URL', edge_name]].explode(edge_name)
    #df_source = df_source.dropna(subset=[edge_name]) # drop empty edge_name, such as affiliation, relatives and Comics
    df_target = df_source.copy()

    edge_list = df_source.merge(df_target, on=edge_name)
    # remove self-loop edges
    index_names = edge_list[edge_list.URL_x == edge_list.URL_y].index
    edge_list.drop(index_names, inplace = True)

    # calculate weight of each edge based on the number of affiliations
    weight_list = edge_list.groupby(['Id_x', 'Id_y']).count()[edge_name].reset_index()
    weight_list = weight_list.rename(columns={edge_name: "weight"})
    #weight_list['weight'] = weight_list['weight'].apply(weight_function, kernel_width=10)
    #print(len(edge_list), len(weight_list))
    edge_df = weight_list.merge(edge_list, on=['Id_x', 'Id_y'], how='inner')

    return edge_df

def check_symmetric(adjacency):
    """
    Recieves a matrix (in this case it will be the adjacency matrix) and prints
    if the given matrix is symmetric or not.

    Inputs
      adjacency [numpy.array] : Matrix to be checked.
    """
    num_non_symmetric = np.count_nonzero((adjacency != adjacency.T).astype(int))

    if num_non_symmetric == 0:
      print('This matrix is symmetric!')
    else:
      print('This matrix is not symmetric!')

def create_adjacency_matirx(df):
    """
    Recieves a dataframe which is already processed in a way that each row
    is an edge connecting a pair of characters and computes its asjacency matrix
    based on the weight (column) of the edge (row).

    Inputs
      df [pandas.DataFrame] : Dataframe computed using get_edge_dataframe function
    
    Outputs
      adjacency [numpy.array] : Adjacency matrix
    """
    n_nodes = df.Id_x.max() + 1
    adjacency = np.zeros((n_nodes, n_nodes), dtype=int)

    for index, row in df.iterrows():
        if np.isnan(row.Id_x) or np.isnan(row.Id_y):
            continue
        
        i, j, w = int(row.Id_x), int(row.Id_y), int(row.weight)
        adjacency[i, j] = w

    return adjacency

def normalize_adjacency_matrix(A):
    """
    Min-Max normalization of the entries of the adjacency matrix to the [1,2] range

    Inputs 
      A [numpy.array] : Adjacency matrix
    Outputs
      A [numpy.array] : Normalized adjacency matrix
    """
    A_nonzero = A[A != 0]
    A[A != 0] = ((A_nonzero - A_nonzero.min()) / (A_nonzero.max() - A_nonzero.min())) + 1

    return A

def set_nodes_attributes(df, graph):
    """
    Given a graph it sets the attributes for each of the graph's nodes. Since in 
    this case the nodes of our graph represent character in the MC or DC universe
    the attributes of each node will be their URL, Real Name, Current Alias, Good
    and nb_appearences
    """
    nodes = df[['URL', 'Real Name', 'Current Alias', 'Good', 'nb_appearences']]
    node_props = nodes.to_dict()

    for key in node_props:
        nx.set_node_attributes(graph, node_props[key], key)

def visualisation_download(adjacency, adjacency_name, attributes, save_path):
    """
    Recieves an adjacency matrix that will firstly convert to a graph, set its 
    attributes and then download it as a gexf file
    
    Inputs 
      adjacency [numpy.array] : Adjacency matrix
      adjacency_name [str] : How to name the gexf file
      attributes [pandas.DataFrame] : Dataframe containing the attributes of each node
      save_path [str] : Path to the folder where the gexf files should be downloaded to
    """
    graph = nx.from_numpy_array(adjacency)
    set_nodes_attributes(attributes, graph)
    nx.write_gexf(graph, save_path+'/{}.gexf'.format(adjacency_name))

def plot_graphs(graph_name, load_path):
    """
    Display the images (already saved in a folder) for each graph

    Inputs
      graph_name [str] : Should be one of the following ["marvel_2000", "marvel_2001"
                                                         "DC_2000", "DC_2001"]
      load_path [str] : Path to the location of the folder where the images are 
                        stored
    """
    fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=("Affiliation Graph", "Relatives Graph"))

    im1 = Image.open(load_path + "/{}_aff.png".format(graph_name))
    im2 = Image.open(load_path + "/{}_rel.png".format(graph_name))
    fig.add_trace(px.imshow(im1).data[0],row=1, col=1)
    fig.add_trace(px.imshow(im2).data[0],row=1, col=2)
    fig.show()
    del im1, im2

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Comics Graph", "Final Graph"))
    im3 = Image.open(load_path + "/{}_com.png".format(graph_name))
    im4 = Image.open(load_path + "/{}_whole.png".format(graph_name))
    fig.add_trace(px.imshow(im3).data[0],row=1, col=1)
    fig.add_trace(px.imshow(im4).data[0],row=1, col=2)
    fig.show()

## Network creation for each dataset

For this part, we get the edges respectively based on Affiliation, Relatives and Comics. Then, we also normalize the adjacency matrices to range $[1, 2]$. We use Min-max normalization. We use the range $[1, 2]$ to make sure the smallest edge weight in original adjacency matrix is at least $1$. The formula is as following:

$$
A_{norm} = \frac{A - A_{min}}{A_{max} - A_{min}} + 1
$$

where $A_{norm}$ is the normalized adjacency matrix, $A$ is the original adjacency matrix.

We also set the weights of Relatives adjacency matrix as $0.5$ and Comics adjacency matrix as $0.1$, because the Affiliation adjacency matrix can provide the most meaningful information. Good characters always have common affiliations. Also, bad characters have common affiliations. Relatives adjacency matrix can provide meaningful information about the relationships between characters. Comics provide the least information to determine whether a character is good, neutral or bad. We concluded to this information after trying different numbers, using just 1 adjacency matrix and performing the classification tasks at the end of this report.

The formula is as following:

$$
A_{all} = A_{affiliation} + 0.5 * A_{relatives} + 0.1 * A_{comics}
$$ 

**Create network for marvel 2000**

In [ ]:
# Creating dataframe whose rows connect a pair of characters sharing the same affiliation
df_edge_list_mar_2000_aff = get_edge_dataframe(marvel_data_2000, edge_name='Affiliation')
# Computing the adjacency matrix
adjacency_mar_2000_aff = create_adjacency_matirx(df_edge_list_mar_2000_aff)
# Normalizing the adjacency matrix
adjacency_mar_2000_aff = normalize_adjacency_matrix(adjacency_mar_2000_aff)

# Creating dataframe whose rows connect a pair of characters sharing the same relative
df_edge_list_mar_2000_rel = get_edge_dataframe(marvel_data_2000, edge_name='Relatives')
# Computing the adjacency matrix
adjacency_mar_2000_rel = create_adjacency_matirx(df_edge_list_mar_2000_rel)
# Normalizing the adjacency matrix
adjacency_mar_2000_rel = normalize_adjacency_matrix(adjacency_mar_2000_rel)

# Creating dataframe whose rows connect a pair of characters sharing the same comic 
df_edge_list_mar_2000_com = get_edge_dataframe(marvel_data_2000, edge_name='Comics')
# Computing the adjacency matrix
adjacency_mar_2000_com = create_adjacency_matirx(df_edge_list_mar_2000_com)
# Normalizing the adjacency matrix
adjacency_mar_2000_com = normalize_adjacency_matrix(adjacency_mar_2000_com)

# Addition of the matrices according to the previously discussed formula 
adjacency_mar_2000 = adjacency_mar_2000_aff + 0.5 * adjacency_mar_2000_rel + 0.1 * adjacency_mar_2000_com

# Creating the graph based on the final adjacency matrix
graph_mar_2000 = nx.from_numpy_array(adjacency_mar_2000)

# Setting the node attributes for our graph
set_nodes_attributes(marvel_data_2000, graph_mar_2000)

In [ ]:
graph_mar_2000.nodes[0]

In [ ]:
visualisation_download(adjacency_mar_2000_aff, "adjacency_mar_2000_aff", marvel_data_2000, gexf_path)
visualisation_download(adjacency_mar_2000_rel, "adjacency_mar_2000_rel", marvel_data_2000, gexf_path)
visualisation_download(adjacency_mar_2000_com, "adjacency_mar_2000_com", marvel_data_2000, gexf_path)
visualisation_download(adjacency_mar_2000, "adjacency_mar_2000", marvel_data_2000, gexf_path)

In [ ]:
plot_graphs("marvel_2000", images_path)

For a more detailed view of the final graph for Marvel 2000 you can visit [our interactive page](https://potamitisn.github.io/NML_Project/Networks/Marvel_2000/)

**Create network for marvel 2001**

In [ ]:
df_edge_list_mar_2001_aff = get_edge_dataframe(marvel_data_2001, edge_name='Affiliation')
adjacency_mar_2001_aff = create_adjacency_matirx(df_edge_list_mar_2001_aff)
adjacency_mar_2001_aff = normalize_adjacency_matrix(adjacency_mar_2001_aff)

df_edge_list_mar_2001_rel = get_edge_dataframe(marvel_data_2001, edge_name='Relatives')
adjacency_mar_2001_rel = create_adjacency_matirx(df_edge_list_mar_2001_rel)
adjacency_mar_2001_rel = normalize_adjacency_matrix(adjacency_mar_2001_rel)

df_edge_list_mar_2001_com = get_edge_dataframe(marvel_data_2001, edge_name='Comics')
adjacency_mar_2001_com = create_adjacency_matirx(df_edge_list_mar_2001_com)
adjacency_mar_2001_com = normalize_adjacency_matrix(adjacency_mar_2001_com)

adjacency_mar_2001 = adjacency_mar_2001_aff + 0.5 * adjacency_mar_2001_rel + 0.1 * adjacency_mar_2001_com

graph_mar_2001 = nx.from_numpy_array(adjacency_mar_2001)

set_nodes_attributes(marvel_data_2001, graph_mar_2001)

In [ ]:
graph_mar_2001.nodes[0]

In [ ]:
visualisation_download(adjacency_mar_2001_aff, "adjacency_mar_2001_aff", marvel_data_2001, gexf_path)
visualisation_download(adjacency_mar_2001_rel, "adjacency_mar_2001_rel", marvel_data_2001, gexf_path)
visualisation_download(adjacency_mar_2001_com, "adjacency_mar_2001_com", marvel_data_2001, gexf_path)
visualisation_download(adjacency_mar_2001, "adjacency_mar_2001", marvel_data_2001, gexf_path)

In [ ]:
plot_graphs("marvel_2001", images_path)

For a more detailed view of the final graph for Marvel 2001 you can visit [our interactive page](https://potamitisn.github.io/NML_Project/Networks/Marvel_2001/)

**Create network for DC 2000**

In [ ]:
df_edge_list_DC_2000_aff = get_edge_dataframe(DC_data_2000, edge_name='Affiliation')
adjacency_DC_2000_aff = create_adjacency_matirx(df_edge_list_DC_2000_aff)
adjacency_DC_2000_aff = normalize_adjacency_matrix(adjacency_DC_2000_aff)

df_edge_list_DC_2000_rel = get_edge_dataframe(DC_data_2000, edge_name='Relatives')
adjacency_DC_2000_rel = create_adjacency_matirx(df_edge_list_DC_2000_rel)
adjacency_DC_2000_rel = normalize_adjacency_matrix(adjacency_DC_2000_rel)

df_edge_list_DC_2000_com = get_edge_dataframe(DC_data_2000, edge_name='Comics')
adjacency_DC_2000_com = create_adjacency_matirx(df_edge_list_DC_2000_com)
adjacency_DC_2000_com = normalize_adjacency_matrix(adjacency_DC_2000_com)

adjacency_DC_2000 = adjacency_DC_2000_aff + 0.5 * adjacency_DC_2000_rel + 0.1 * adjacency_DC_2000_com

graph_DC_2000 = nx.from_numpy_array(adjacency_DC_2000)

set_nodes_attributes(DC_data_2000, graph_DC_2000)

In [ ]:
graph_DC_2000.nodes[0]

In [ ]:
visualisation_download(adjacency_DC_2000_aff, "adjacency_DC_2000_aff", DC_data_2000, gexf_path)
visualisation_download(adjacency_DC_2000_rel, "adjacency_DC_2000_rel", DC_data_2000, gexf_path)
visualisation_download(adjacency_DC_2000_com, "adjacency_DC_2000_com", DC_data_2000, gexf_path)
visualisation_download(adjacency_DC_2000, "adjacency_DC_2000", DC_data_2000, gexf_path)

In [ ]:
plot_graphs("DC_2000", images_path)

For a more detailed view of the final graph for DC 2000 you can visit [our interactive page](https://potamitisn.github.io/NML_Project/Networks/DC_2000/)

**Create network for DC 2001**

In [ ]:
df_edge_list_DC_2001_aff = get_edge_dataframe(DC_data_2001, edge_name='Affiliation')
adjacency_DC_2001_aff = create_adjacency_matirx(df_edge_list_DC_2001_aff)
adjacency_DC_2001_aff = normalize_adjacency_matrix(adjacency_DC_2001_aff)

df_edge_list_DC_2001_rel = get_edge_dataframe(DC_data_2001, edge_name='Relatives')
adjacency_DC_2001_rel = create_adjacency_matirx(df_edge_list_DC_2001_rel)
adjacency_DC_2001_rel = normalize_adjacency_matrix(adjacency_DC_2001_rel)

df_edge_list_DC_2001_com = get_edge_dataframe(DC_data_2001, edge_name='Comics')
adjacency_DC_2001_com = create_adjacency_matirx(df_edge_list_DC_2001_com)
adjacency_DC_2001_com = normalize_adjacency_matrix(adjacency_DC_2001_com)

adjacency_DC_2001 = adjacency_DC_2001_aff + 0.5 * adjacency_DC_2001_rel + 0.1 * adjacency_DC_2001_com

graph_DC_2001 = nx.from_numpy_array(adjacency_DC_2001)

set_nodes_attributes(DC_data_2001, graph_DC_2001)

In [ ]:
visualisation_download(adjacency_DC_2001_aff, "adjacency_DC_2001_aff", DC_data_2001, gexf_path)
visualisation_download(adjacency_DC_2001_rel, "adjacency_DC_2001_rel", DC_data_2001, gexf_path)
visualisation_download(adjacency_DC_2001_com, "adjacency_DC_2001_com", DC_data_2001, gexf_path)
visualisation_download(adjacency_DC_2001, "adjacency_DC_2001", DC_data_2001, gexf_path)

In [ ]:
graph_DC_2001.nodes[0]

In [ ]:
plot_graphs("DC_2001", images_path)

For a more detailed view of the final graph for DC 2001 you can visit [our interactive page](https://potamitisn.github.io/NML_Project/Networks/DC_2001/)

**Save adjacency matrices**

Computing the adjacency matrices takes about 1 minute for each dataset so at this point we can save them. Next time we need them, we can load them directly.

In [ ]:
np.save(os.path.join(graphs_path, "adjacency_maxtirx_marvel_2000.npy"), adjacency_mar_2000)
np.save(os.path.join(graphs_path, "adjacency_maxtirx_marvel_2001.npy"), adjacency_mar_2001)
np.save(os.path.join(graphs_path, "adjacency_maxtirx_DC_2000.npy"), adjacency_DC_2000)
np.save(os.path.join(graphs_path, "adjacency_maxtirx_DC_2001.npy"), adjacency_DC_2001)

**Load adjacency matrices and Create Graphs**

In case the adjacency matrices are already calculated we can load them directly to speed up the process

In [ ]:
current_path = os.getcwd()
graphs_path = os.path.join(current_path, 'graphs')

adjacency_mar_2000 = np.load(os.path.join(graphs_path, "adjacency_maxtirx_marvel_2000.npy"))
adjacency_mar_2001 = np.load(os.path.join(graphs_path, "adjacency_maxtirx_marvel_2001.npy"))
adjacency_DC_2000 = np.load(os.path.join(graphs_path, "adjacency_maxtirx_DC_2000.npy"))
adjacency_DC_2001 = np.load(os.path.join(graphs_path, "adjacency_maxtirx_DC_2001.npy"))

graph_mar_2000 = nx.from_numpy_array(adjacency_mar_2000)
set_nodes_attributes(marvel_data_2000, graph_mar_2000)

graph_mar_2001 = nx.from_numpy_array(adjacency_mar_2001)
set_nodes_attributes(marvel_data_2001, graph_mar_2001)

graph_DC_2000 = nx.from_numpy_array(adjacency_DC_2000)
set_nodes_attributes(DC_data_2000, graph_DC_2000)

graph_DC_2001 = nx.from_numpy_array(adjacency_DC_2001)
set_nodes_attributes(DC_data_2001, graph_DC_2001)

# Exploratory Data Analysis

In this section we will perform some EDA to get a precise understanding of the graphs that we are dealing with.

## Labels distribution

We will begin by taking a look at the number of characters classified as `Good`, `Bad`or `Neutral`

In [ ]:
def plotLabelDistribution(df, axIndex, name='Marvel 2000'):
    """
    Plots a barplot with the number of good, bad and neutral characters
    in our dataset

    Inputs 
      df [pandas.DataFrame] : Dataset dataframe
      ax [matplotlib.axes._subplots.AxesSubplot] : Position of the graph
      name [str] : Name of the graph to plot. Can be one of the following 
                   ["Marvel 2000", "Marvel 2001", "DC 2000", "DC 2001"]
    """
    labels = df['Good'].value_counts()
    labels = labels.rename(index={1: 'Good', 0:'Neutral', -1:'Bad'})
    ax = sns.barplot(x=labels.index, y=labels.values, ax=axIndex)
    ax.set_title('{} labels distribution'.format(name), fontweight="bold", fontsize=15)
    ax.set_ylabel('Number', fontweight="bold", fontsize=15)
    ax.set_xlabel('Class', fontweight="bold", fontsize=15)

In [ ]:
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
plotLabelDistribution(marvel_data_2000, axes[0][0], name='Marvel 2000')
plotLabelDistribution(marvel_data_2001, axes[0][1], name='Marvel 2001')
plotLabelDistribution(DC_data_2000, axes[1][0], name='DC 2000')
plotLabelDistribution(DC_data_2000, axes[1][1], name='DC 2001')
plt.savefig(os.path.join(figures_path, 'label_distribution_node.png'))

From above figures, we can see the labels distributions of all datasets are **unbalanced**, especially in the DC case. Therefore, to do the node classification task, we can not just consider the accuracy. We should also consider the **F1 score and confusion matrix.**

Apart from that, we also need to split the dataset into train set and test set in **stratified fashion**, which makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to the **labels distributions**.

## Sparsity pattern of the adjacency matrices

Next, we will take a look into the sparsity patter of the final adjacency matrix for each dataset.

In [ ]:
# Sparsity for Marvel 2000
sort_mar_2000 = np.argsort(adjacency_mar_2000.sum(1))
adjacency_mar_2000_sort = adjacency_mar_2000[sort_mar_2000,:][:,sort_mar_2000]

# Sparsity for Marvel 2001
sort_mar_2001 = np.argsort(adjacency_mar_2001.sum(1))
adjacency_mar_2001_sort = adjacency_mar_2001[sort_mar_2001,:][:,sort_mar_2001]

# Sparsity for DC 2000
sort_DC_2000 = np.argsort(adjacency_DC_2000.sum(1))
adjacency_DC_2000_sort = adjacency_DC_2000[sort_DC_2000,:][:,sort_DC_2000]

# Sparsity for DC 2001
sort_DC_2001 = np.argsort(adjacency_DC_2001.sum(1))
adjacency_DC_2001_sort = adjacency_DC_2001[sort_DC_2001,:][:,sort_DC_2001]

Plotting our results.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes[0][0].set_title('Marvel 2000 graph: adjacency matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[0][0].spy(adjacency_mar_2000_sort)
axes[0][1].set_title('Marvel 2001 graph: adjacency matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[0][1].spy(adjacency_mar_2001_sort)
axes[1][0].set_title('DC 2000 graph: adjacency matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[1][0].spy(adjacency_DC_2000_sort)
axes[1][1].set_title('DC 2001 graph: adjacency matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[1][1].spy(adjacency_DC_2001_sort)

plt.savefig(os.path.join(figures_path, 'sparsity_patterns.png'))

From the above figures, we can see the graphs are very sparse. Not all characters are connected. 

## Degree Distribution and Moments

Moving on, we will compute the degree distribution and moments for each graph/adjacency.

### Degree distribution

In [ ]:
# Degree for Marvel 2000
degree_mar_2000 = adjacency_mar_2000.sum(1)
deg_hist_norm_mar_2000 = np.ones(adjacency_mar_2000.shape[0]) / adjacency_mar_2000.sum()

# Degree for Marvel 2001
degree_mar_2001 = adjacency_mar_2001.sum(1)
deg_hist_norm_mar_2001 = np.ones(adjacency_mar_2001.shape[0]) / adjacency_mar_2001.sum()

# Degree for DC 2000
degree_DC_2000 = adjacency_DC_2000.sum(1)
deg_hist_norm_DC_2000 = np.ones(adjacency_DC_2000.shape[0]) / adjacency_DC_2000.sum()

# Degree for DC 2001
degree_DC_2001 = adjacency_DC_2001.sum(1)
deg_hist_norm_DC_2001 = np.ones(adjacency_DC_2001.shape[0]) / adjacency_DC_2001.sum()

Plotting our results.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

sns.histplot(ax=axes[0][0], x=degree_mar_2000, weights=deg_hist_norm_mar_2000, bins=20, kde=True)
axes[0][0].set_title('Marvel 2000 graph degree distribution', fontweight="bold", fontsize=15)
axes[0][0].set_xlabel('Degree', fontweight="bold", fontsize=15)
axes[0][0].set_ylabel('Normalized count', fontweight="bold", fontsize=15)

sns.histplot(ax=axes[0][1], x=degree_mar_2001, weights=deg_hist_norm_mar_2001, bins=20, kde=True)
axes[0][1].set_title('Marvel 2001 degree distribution', fontweight="bold", fontsize=15)
axes[0][1].set_xlabel('Degree', fontweight="bold", fontsize=15)
axes[0][1].set_ylabel('Normalized count', fontweight="bold", fontsize=15)

sns.histplot(ax=axes[1][0], x=degree_DC_2000, weights=deg_hist_norm_DC_2000, bins=20, kde=True)
axes[1][0].set_title('DC 2000 graph degree distribution', fontweight="bold", fontsize=15)
axes[1][0].set_xlabel('Degree', fontweight="bold", fontsize=15)
axes[1][0].set_ylabel('Normalized count', fontweight="bold", fontsize=15)

sns.histplot(ax=axes[1][1], x=degree_DC_2001, weights=deg_hist_norm_DC_2001, bins=20, kde=True)
axes[1][1].set_title('DC 2001 degree distribution', fontweight="bold", fontsize=15)
axes[1][1].set_xlabel('Degree', fontweight="bold", fontsize=15)
axes[1][1].set_ylabel('Normalized count', fontweight="bold", fontsize=15)

plt.savefig(os.path.join(figures_path, 'degree_distributions.png'))

### Moments

In [ ]:
def first_second_moment(A, name='Marvel 2000'):
    """
    Recieves an adjacency matrix for which it computes and then prints
    the first and second moment.

    Inputs
      A [numpy.array] : Adjacency matrix
      name [str] : Name of the Adjacency matrix to include at the prit
                   (just for the sake of clarity)
    """
    degree = A.sum(1)
    values, counts = np.unique(degree, return_counts=True)
    
    probs = counts / counts.sum()

    moment_1 = (values * probs).sum()
    moment_2 = (values * values * probs).sum()
    
    print('First moment for {} is : {:.2f}'.format(name, moment_1))
    print('Second moment for {} is : {:.2f}\n'.format(name, moment_2))

In [ ]:
first_second_moment(adjacency_mar_2000, name='Marvel 2000')
first_second_moment(adjacency_mar_2001, name='Marvel 2001')
first_second_moment(adjacency_DC_2000, name='DC 2000')
first_second_moment(adjacency_DC_2001, name='DC 2001')

**Comment:**

From above figures, we can see all the degree distributions are based on power law (or fat-tailed distribution). This is the **social network's behaviour**.

Apart from that, we notice that the second moment, $\left\langle k^{2}\right\rangle$ is large. We also notice there are some hubs (high-degree nodes). Therefore, this is the scale-free behaviour.

### Total degree

In [ ]:
def total_degree_print(A, name='Marvel 2000'):
    edges = A.sum()
    print('Total degree for {} is : {:.1f}\n'.format(name, edges))

In [ ]:
total_degree_print(adjacency_mar_2000, name='Marvel 2000')
total_degree_print(adjacency_mar_2001, name='Marvel 2001')
total_degree_print(adjacency_DC_2000, name='DC 2000')
total_degree_print(adjacency_DC_2001, name='DC 2001')

From above printings, we can see Marvel graphs have more edges than those of DC datasets. Therefore, more characters in Marvel universe are connected. 

## Important Nodes

To verify our network creation is meaningful, we visulaize the top 10 important nodes based on **Degree** and **Betweenness Centrality.** The details of the two global properties are shown in the following:

**Degree:** In this project, this is simply the normalized number of connections the node has in the network. In the Marvel and DC universe, this corresponds to the total number of common affiliations, relatives and comics which the two characters have. For example, imagine if you are Iron man, you will be very important because you will have a lot of common affiliations and relatives with other characters. You also appear in a lot of comics.

**Betweenness Centrality:** this corresponds to how many shortest paths in the network lead through the node. For example, imagine you are Iron man and you want to send a message to Wolverine. The shortest path how to send it is via Spider man, because he interacted both with Iron man and Wolverine. On the other side, if you want to send a message to Captain America, you don’t have to go through Spider man because Iron man knows Captain America directly. The betweenness centrality for Spider man is computed using the number of shortest paths between all other characters that pass through him.

In [ ]:
def plotNodeDegree(graph, axIndex, name='Marvel 2000'):
    """
    Plots top 10 most importand nodes of a given graph based on degree 

    Inputs
      graph [networkx.classes.graph.Graph] : Graph to be inspected
      axIndex [matplotlib.axes._subplots.AxesSubplot] : Position of the graph
      name [str] : Name of the graph to plot (just used for the title). Should be
                   one of the following : ["Marvel 2000", "Marvel 2001", "DC 2000", "DC 2001"]
    """
    topNode = sorted(graph.degree(weight='weight'), key=lambda x: x[1], reverse=True)[:10]
    deg = [node[1] for node in topNode]
    alias = [graph.nodes[node[0]]['Current Alias'] if graph.nodes[node[0]]['Current Alias'] != '' else graph.nodes[node[0]]['Real Name'] for node in topNode]

    ax = sns.barplot(x=deg, y=alias, ax=axIndex, ci=None)
    ax.set_title('{} Top 10 Node Degree'.format(name), fontweight="bold", fontsize=15)
    ax.set_ylabel('Current Alias', fontweight="bold", fontsize=15)
    ax.set_xlabel('Degree', fontweight="bold", fontsize=15)

In [ ]:
def plotNodeBetween(graph, axIndex, name='Marvel 2000'):
    """
    Plots top 10 most importand nodes of a given graph based on betwenness centrality 

    Inputs
      graph [networkx.classes.graph.Graph] : Graph to be inspected
      axIndex [matplotlib.axes._subplots.AxesSubplot] : Position of the graph
      name [str] : Name of the graph to plot (just used for the title). Should be
                   one of the following : ["Marvel 2000", "Marvel 2001", "DC 2000", "DC 2001"]
    """
    topNode = sorted(nx.betweenness_centrality(graph, weight=None).items(), key=lambda x: x[1], reverse=True)[:10]
    betweenness_centrality = [node[1] for node in topNode]
    alias = [graph.nodes[node[0]]['Current Alias'] if graph.nodes[node[0]]['Current Alias'] != '' else graph.nodes[node[0]]['Real Name'] for node in topNode]

    ax = sns.barplot(x=betweenness_centrality, y=alias, ax=axIndex, ci=None)
    ax.set_title('{} Top 10 Node Betweenness Centrality'.format(name), fontweight="bold", fontsize=15)
    ax.set_ylabel('Current Alias', fontweight="bold", fontsize=15)
    ax.set_xlabel('Degree', fontweight="bold", fontsize=15)

### Top 10 Important nodes based on Degree


In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.5)
fig, axes = plt.subplots(2, 2, figsize=(39, 15))
plotNodeDegree(graph_mar_2000, axes[0][0], name='Marvel 2000')
plotNodeDegree(graph_mar_2001, axes[0][1], name='Marvel 2001')
plotNodeDegree(graph_DC_2000, axes[1][0], name='DC 2000')
plotNodeDegree(graph_DC_2001, axes[1][1], name='DC 2001')

plt.savefig(os.path.join(figures_path, 'top_nodes_degree.png'))

### Top 10 importand nodes based on betwenness centrality

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.5)
fig, axes = plt.subplots(2, 2, figsize=(43, 15))
plotNodeBetween(graph_mar_2000, axes[0][0], name='Marvel 2000')
plotNodeBetween(graph_mar_2001, axes[0][1], name='Marvel 2001')
plotNodeBetween(graph_DC_2000, axes[1][0], name='DC 2000')
plotNodeBetween(graph_DC_2001, axes[1][1], name='DC 2001')

plt.savefig(os.path.join(figures_path, 'top_nodes_betweenness.png'))

From above figures, we can see all important characters meet our expectations. For example, Iron man and Spider man are very popular in Marvel universe. Also, Superman and Batman are very popular in DC universe.

## Path matrix (N = 10)

Moving forward we will compute the path matrix as a mean of measurement concerning the connectivity of the graphs. The path matrix is defined by :

$$
 P_{ij} = \displaystyle\sum_{k=0}^{N}C_{k}(i,j). 
$$

In [ ]:
def path_matrix(A, n):
    """
    Given an adjacency matrix, it computes and returns its path matrix for N=n

    Inputs :
      A [numpy.array] : Adjacency matrix
      n [int] : Power of up to which the path matrix is computed. Corresponds to
                N from the above formula
    Outputs
      path_matrix [numpy.array] : Computed path matrix
    """
    path_lengths = range(11)
    path_matrix = sum([np.linalg.matrix_power(A, k) for k in path_lengths])

    return path_matrix

In [ ]:
# Computing path matrix for Marvel 2000
path_mar_2000 = path_matrix(adjacency_mar_2000, n=10)

#Computing path matrix for Marvel 2001
path_mar_2001 = path_matrix(adjacency_mar_2001, n=10)

#Computing path matrix for DC 2000
path_DC_2000 = path_matrix(adjacency_DC_2000, n=10)

#Computing path matrix for DC 2001
path_DC_2001 = path_matrix(adjacency_DC_2001, n=10)

Plotting our results :

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes[0][0].set_title('Marvel 2000 graph: path matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[0][0].spy(path_mar_2000)
axes[0][1].set_title('Marvel 2001 graph: path matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[0][1].spy(path_mar_2001)
axes[1][0].set_title('DC 2000 graph: path matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[1][0].spy(path_DC_2000)
axes[1][1].set_title('DC 2001 graph: path matrix sparsity pattern', fontweight="bold", fontsize=15)
axes[1][1].spy(path_DC_2001)

plt.savefig(os.path.join(figures_path, 'path_matrix.png'))

From above figures, we can see once more that the Marvel graphs are more connected than DC graphs.

## Connected compoents, giant components and clustering coefficients

In this section we will study the connected componets, the giant component and the average clustering coefficient of the graphs. The average clustering coefficient is given by : 

$$
C = \frac{1}{N}∑_{i=1}^NC_i 
$$

where $C_i$ is the clustering coefficient of each node.

In [ ]:
def print_connected_giant_clustering(graph, name='Marvel 2000'):
    """
    Given a graph it prints some statements concering the connected components
    of the graph and its average clustering coefficient

    Inputs 
      graph [networkx.classes.graph.Graph] : Graph to be inspected
      name [str] : Name of the graph to be included in the print (just for clarity)
    """
    giant_com = graph.subgraph(max(nx.connected_components(graph), key=len)).copy()
    n_connected_com = nx.number_connected_components(graph)
    average_clustering_coe = nx.average_clustering(graph)
    
    print('{} has {} nodes and {:.1f} edges/total weight'.format(name, graph.number_of_nodes(), graph.size(weight='weight')))
    print('{} number of connected components: {}'.format(name, n_connected_com))
    print('The giant component of {} with diameter {} has {} nodes and {:.1f} edges/total weight.'.format(name, nx.diameter(giant_com), giant_com.number_of_nodes(), giant_com.size(weight='weight')))
    print('{} average clustering coefficient: {:.3f}\n'.format(name, average_clustering_coe))

In [ ]:
print_connected_giant_clustering(graph_mar_2000, name='Marvel 2000')
print_connected_giant_clustering(graph_mar_2001, name='Marvel 2001')
print_connected_giant_clustering(graph_DC_2000, name='DC 2000')
print_connected_giant_clustering(graph_DC_2001, name='DC 2001')

From above printings, we can see DC universe has more connected components that Marvel universe. Therefore, DC universe does not want to connect all characters. DC universe probably would like to make stories about some characters not all characters. 

We can also see Marvel universe and DC universe have the similar average clustering coefficients. Clustering coefficient can show the connectivity pattern of a node to its neighbors. Relatively large values are observed for the networks having more regular connectivity patterns. From above printings, we find the clustering coeficients of the Marvel and DC graphs are large. Therefore, the two graphs are not random networks. 

## Feature extraction

For feature extraction, we consider following five types of features:

**Hand-crafted features:**
*   Degree
*   Betweenness centrality
*   Closeness centrality
*   Eigenvector centrality
*   Clustering coefficients

We choose above five hand-crafted features for the following reasons : 

- Firstly, based on the exploratory data analysis, we see node degree and betweenness centrality provide meaningful information about the important nodes (please see the discussion in Exploratory Data Analysis). 

- Secondly, closeness centrality measures the importance of a node based on how many shortest path lengths to all other nodes. For example, Iron man can interact with other **good** characters directly. Iron man can have common affiliations and relatives with good characters such as Spider man and Captain America. Eigenvector centrality corresponds to a node surrounded by how many important neighbors. For example, Mentor is a good character in Marvel, but he is not very popular. Mentor can be surrounded by important good characters.

- Thirdly, we use clustering coefficients to provide information about the subgraph containing the neighbors of a node, and all edges between nodes in its neighborhood. This can measure about the connectivity pattern of a node to its neighbors and provide information about the network model.

**More flexible node representations:**

**Node2vec** to provide 30 dimensional feature vector for each node.

We use Node2vec (with $p = 1.0$ and $q = 10.0$) to characterize the local view of the network. We notice in social-network behavioural network, the communities of nodes are not very clear. Therefore, we use Node2vec (with $p = 1.0$ and $q = 10.0$) for capturing structural nodes, e.g., hubs, outliers.

**Labels**
For labels of nodes, we use the `Good` feature of nodes (Integer value representing whether the character is good (+1), neutral (0), or evil (-1)). We add one to the interger values, so the character is good (+2), neutral (1), or evil (0). This can help us train graph nerual networks with cross-entropy loss.

In [ ]:
def Node2Vec(G, dimensions, walk_length, num_walks, p=1.0, q=1.0):
    """
    Creates additional dimeansional feature vector for each vector

    Inputs
      G [networkx.classes.graph.Graph] : Graph to be inspected
      dimensions [int]: Embedding dimensions
      walk_length [int]: Maximum length of each random walk
      num_walks [int]: Total number of random walks per root node
      p [float]:
      q [float]: 
    """
    seed = 0
    rw = BiasedRandomWalk(G)
    walks = rw.run(
        nodes=G.nodes(),
        length=walk_length,
        n=num_walks,
        p=p,
        q=q,
        seed=seed
    )
    str_walks = [[str(n) for n in walk] for walk in walks]
    model = Word2Vec(str_walks, size=dimensions, window=5, min_count=0, sg=1, seed=seed, workers=1)

    return model

In [ ]:
def extract_features(G, nodewalk_dim=30):
  """

  Inputs
    G [networkx.classes.graph.Graph] : Graph to be inspected
    nodewalk_dim [int]: Embedding dimensions for Node2Vec function
  
  Outputs
    Features [numpy.array] : NxD where N is the number of nodes and D(=35) are the
                             features created by this function 
  """
  ## degree
  degrees = G.degree(weight='weight')
  degree_feature = np.array([degrees[node] for node in G.nodes()])
  
  ## 3 node centrality measures
  betweenness_centrality = nx.betweenness_centrality(G, weight=None)
  closeness_centrality = nx.closeness_centrality(G, distance=None)
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=500, weight=None)
  betweenness_centrality_feature = np.array([betweenness_centrality[node] for node in G.nodes()])
  closeness_centrality_feature = np.array([closeness_centrality[node] for node in G.nodes()])
  eigenvector_centrality_feature = np.array([eigenvector_centrality[node] for node in G.nodes()])

  ## clustering coefficient of each node
  clustering_coefficients = nx.clustering(G, weight=None)
  clustering_coefficient_feature = np.array([clustering_coefficients[node] for node in G.nodes()])

  ## Node2walk
  model = Node2Vec(StellarGraph.from_networkx(G, edge_weight_attr=None), dimensions=nodewalk_dim, walk_length=10, num_walks=50, p=1.0, q=10.0)
  features_n2v1 = np.array([model.wv.get_vector(str(node)) for node in G.nodes()])
  
  ## stack the features
  Features = np.concatenate((degree_feature[:, np.newaxis], betweenness_centrality_feature[:, np.newaxis], 
                             closeness_centrality_feature[:, np.newaxis], eigenvector_centrality_feature[:, np.newaxis], clustering_coefficient_feature[:, np.newaxis], features_n2v1), axis=1)

  return Features

### Extracting the features

In the following code blocs we will extract the features described so far. For each dataset it takes about 3 minutes so for time reasons we saved them already. If you want you can skip the next 5 code blocks and load them direclty with the provided code.

**Extract features for Marvel 2000 dataset**

In [ ]:
features_mar_2000 = extract_features(graph_mar_2000)
targets_mar_2000 = np.array([nx.get_node_attributes(graph_mar_2000, "Good")[node]+1 for node in graph_mar_2000.nodes()])

**Extract features for Marvel 2001 dataset**

In [ ]:
features_mar_2001 = extract_features(graph_mar_2001)
targets_mar_2001 = np.array([nx.get_node_attributes(graph_mar_2001, "Good")[node]+1 for node in graph_mar_2001.nodes()])

**Extract features for DC 2000 dataset**

In [ ]:
features_DC_2000 = extract_features(graph_DC_2000)
targets_DC_2000 = np.array([nx.get_node_attributes(graph_DC_2000, "Good")[node]+1 for node in graph_DC_2000.nodes()])

**Extract features for 2001 dataset**

In [ ]:
features_DC_2001 = extract_features(graph_DC_2001)
targets_DC_2001 = np.array([nx.get_node_attributes(graph_DC_2001, "Good")[node]+1 for node in graph_DC_2001.nodes()])

### Save/Load features and labels

Creating the feature and label arrays takes about 3 minutes for each dataset. For this reason we save them to speed up any case where we need them in the future.

In [ ]:
np.save(os.path.join(features_path, "features_marvel_2000.npy"), features_mar_2000)
np.save(os.path.join(features_path, "y_marvel_2000.npy"), targets_mar_2000)

np.save(os.path.join(features_path, "features_marvel_2001.npy"), features_mar_2001)
np.save(os.path.join(features_path, "y_marvel_2001.npy"), targets_mar_2001)

np.save(os.path.join(features_path, "features_DC_2000.npy"), features_DC_2000)
np.save(os.path.join(features_path, "y_DC_2000.npy"), targets_DC_2000)

np.save(os.path.join(features_path, "features_DC_2001.npy"), features_DC_2001)
np.save(os.path.join(features_path, "y_DC_2001.npy"), targets_DC_2001)

**Load features and labels**

In [ ]:
features_mar_2000 = np.load(os.path.join(features_path, "features_marvel_2000.npy"))
targets_mar_2000 = np.load(os.path.join(features_path, "y_marvel_2000.npy"))

features_mar_2001 = np.load(os.path.join(features_path, "features_marvel_2001.npy"))
targets_mar_2001 = np.load(os.path.join(features_path, "y_marvel_2001.npy"))

features_DC_2000 = np.load(os.path.join(features_path, "features_DC_2000.npy")) 
targets_DC_2000 = np.load(os.path.join(features_path, "y_DC_2000.npy"))

features_DC_2001 = np.load(os.path.join(features_path, "features_DC_2001.npy"))
targets_DC_2001 = np.load(os.path.join(features_path, "y_DC_2001.npy"))

## Spilt dataset in train set and test set, Normalize data

In order to be able to evaluate our models we first need to separate our dataset in train dataset (used for training) and test dataset (not used for training). The way to do this network machine learning is to create mask arrays. These arrays have the same dimensions for our feature matrix and label matrix but contain only `True`and `False` values indicating which elements are gonna be used for training and which for testing.

**Train and Test split**

In [ ]:
def split_train_test(features, targets, ratio=0.2, seed=0):
    """
    Given the train matrix and the label vector computes the masks to be used in 
    order to spli the dataset into a train set and training set. An important aspect
    is that we use the stratify option from the train_test_split function. This
    helps us have a balanced training set.

    Inputs
      features [numpy.ndarray]: NxD Train matrix where N is the number of nodes and 
                              D the number of features
      targets [numpy.ndarray]: Nx1 Label vect where each node is labeld with 0,1 or 2
                             depending wether they are good, bad or neutral.
      ratio [float]: Portion of dataset to be used just for testing and not training
      seed [int]: Random seed

    Outputs
      train_mask [numpy.ndarray]: NxD Train mask filled with True/False indicating
                                  which elements are to be used for training or not
      test_mask [numpy.ndarray]: Nx1 Test mask filled with True/False indicating
                                  which elements are to be used for testing or not
    """
    train_ratio = ratio
    n_nodes = features.shape[0]
    n_train = int(n_nodes * train_ratio)
    idx = np.array([i for i in range(n_nodes)])
    
    n_train, n_test, y_train, y_test = train_test_split(idx, targets, stratify=targets, test_size=ratio, random_state = seed)
    train_mask = np.full_like(targets, False, dtype=bool)
    train_mask[idx[n_train]] = True
    test_mask = np.full_like(targets, False, dtype=bool)
    test_mask[idx[n_test]] = True

    train_mask = train_mask
    test_mask = test_mask

    return train_mask, test_mask 

In [ ]:
# Creating the masks for Marvel 2000 dataset
train_mask_mar_2000, test_mask_mar_2000 = split_train_test(features_mar_2000, targets_mar_2000, ratio=0.2, seed=0)

# Creating the masks for Marvel 2001 dataset
train_mask_mar_2001, test_mask_mar_2001 = split_train_test(features_mar_2001, targets_mar_2001, ratio=0.2, seed=0)

# Creating the masks for DC 2000 dataset
train_mask_DC_2000, test_mask_DC_2000 = split_train_test(features_DC_2000, targets_DC_2000, ratio=0.2, seed=0)

# Creating the masks for DC 2001 dataset
train_mask_DC_2001, test_mask_DC_2001 = split_train_test(features_DC_2001, targets_DC_2001, ratio=0.2, seed=0)

**Normalize data**

In [ ]:
def normalize(features, train_mask, test_mask):
    """
    Normalizes the features

    Inputs
      features [numpy.ndarray]: NxD Train matrix where N is the number of nodes and 
                              D the number of features
      train_mask [numpy.ndarray]: NxD Train mask filled with True/False indicating
                                  which elements are to be used for training or not
      test_mask [numpy.ndarray]: Nx1 Test mask filled with True/False indicating
                                  which elements are to be used for testing or not
    Outputs
      normalized_features [numpy.ndarray] : NXD Normalized features matrix
    """
    normalized_features = features.copy()
    X_train, X_test = features[train_mask, :].copy(), features[test_mask, :].copy()
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    normalized_features[train_mask, :] = X_train
    normalized_features[test_mask, :] = X_test
    return normalized_features

In [ ]:
features_mar_2000 = normalize(features_mar_2000, train_mask_mar_2000, test_mask_mar_2000)
features_mar_2001 = normalize(features_mar_2001, train_mask_mar_2001, test_mask_mar_2001)
features_DC_2000 = normalize(features_DC_2000, train_mask_DC_2000, test_mask_DC_2000)
features_DC_2001 = normalize(features_DC_2001, train_mask_DC_2001, test_mask_DC_2001)

**Set features as the attributes of nodes**

In [ ]:
def set_features_attr(features, labels, train_mask, test_mask, graph):
    """
    Updates the node attributes of the grap with the calculated features

    Inputs
      features [numpy.ndarray]: NxD Train matrix where N is the number of nodes and 
                                D the number of features
      labels [numpy.ndarray]: Nx1 Label vect where each node is labeld with 0,1 or 2
                              depending wether they are good, bad or neutral.
      train_mask [numpy.ndarray]: NxD Train mask filled with True/False indicating
                                  which elements are to be used for training or not
      test_mask [numpy.ndarray]: Nx1 Test mask filled with True/False indicating
                                 which elements are to be used for testing or not
      graph [networkx.classes.graph.Graph] : Graph for which we want to update the
                                             attributes

    """
    x_attr = {i:features[i, :] for i in range(features.shape[0])}
    y_attr = {i:labels[i] for i in range(labels.shape[0])}
    train_mask_attr = {i:train_mask[i] for i in range(train_mask.shape[0])}
    test_mask_attr = {i:test_mask[i] for i in range(test_mask.shape[0])}

    nx.set_node_attributes(graph, x_attr, "x")
    nx.set_node_attributes(graph, y_attr, "y")
    nx.set_node_attributes(graph, train_mask_attr, "train_mask")
    nx.set_node_attributes(graph, test_mask_attr, "test_mask")

In [ ]:
set_features_attr(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, graph_mar_2000)
set_features_attr(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, graph_mar_2001)
set_features_attr(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, graph_DC_2000)
set_features_attr(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, graph_DC_2001)

In [ ]:
graph_mar_2000.nodes[0]

## Features Visualization (PCA, Isomap, TSNE)

At this moment we are dealing with high dimensional data (35 features) which means that we can't visualise them in their current form. To do that we will first have to apply some statistical methods. In this section we will use the following methods to visualise our data : 

- Principal Components Analysis (PCA)
-  Isomap
- T-distributed Stochastic Neighbor embedding (TSNE)

In [ ]:
def visualize_features(features, labels, dim=3):
    """
    Inputs
      features [numpy.ndarray]: NxD Train matrix where N is the number of nodes and 
                                D the number of features
      labels [numpy.ndarray]: Nx1 Label vect where each node is labeld with 0,1 or 2
                              depending wether they are good, bad or neutral.
    """
    pca = PCA(n_components=dim)
    tsne = TSNE(n_components=dim)
    isomap = Isomap(n_components=dim,n_neighbors=20)
    
    colors = ["navy", "turquoise", "darkorange"]

    pca_features = pca.fit_transform(features)
    tsne_features = tsne.fit_transform(features)
    isomap_features = isomap.fit_transform(features)
    
    fig = plt.figure(figsize=(18, 7))
   
    index = 1
    for data, name in zip([pca_features, tsne_features, isomap_features], ['PCA', 'TSNE', 'Isomap']):
        ax = fig.add_subplot(1, 3, index, projection='3d')
        for color, i, target_name in zip(colors, [0, 1, 2], ['Bad', 'Neutral', 'Good']):
            ax.scatter(
                data[labels == i, 0], data[labels == i, 1], data[labels == i, 2], color=color, alpha=0.8, lw=2, label=target_name
              )
            ax.legend(loc="best", shadow=False, scatterpoints=1)
            ax.set_xlabel("x1", fontweight='bold')
            ax.set_ylabel("x2", fontweight='bold')
            ax.set_zlabel("x3", fontweight='bold')
            ax.set_title(name, fontsize=16, fontweight='bold', loc='left')
        index += 1 

**Visualising Marvel 2000 feature dataset**

In [ ]:
visualize_features(features_mar_2000, targets_mar_2000)

**Visualising Marvel 2001 feature dataset**

In [ ]:
visualize_features(features_mar_2001, targets_mar_2001)

**Visualising DC 2000 feature dataset**

In [ ]:
visualize_features(features_DC_2000, targets_DC_2000)

**Visualising DC 2001 feature dataset**

In [ ]:
visualize_features(features_DC_2001, targets_DC_2001)

## Graph signal processing and graph Tikhonov regularization

To prevent machine learning models from overfitting on the training set, we do the graph Tikhonov regularization in this part. We also focus on graph Fourier transform and graph signal processing.

### Utility functions 

To achieve this we define a few more utility functions in the following code block :

In [ ]:
def compute_laplacian(adjacency: np.ndarray, normalize: str):
    """
    Computes Laplacian matrix given an adjacency matrix and a normalization method
    
    Inputs
      adjancency [numpy.ndarray]: Adjacency matrix
      normalize [str]: can be None, 'sym' or 'rw' for the combinatorial, symmetric normalized or random walk Laplacians
    Outputs:
        L [numpy.ndarray]: Combinatorial or symmetric normalized Laplacian.
    """
    n = adjacency.shape[0]
    D = np.diag(np.sum(adjacency, axis=1))
    cm_L = D - adjacency

    if normalize == 'sym':
        D_sqrt = np.diag(np.clip(np.sum(adjacency, axis=1), 1, None)**(-1/2))
        L = D_sqrt @ cm_L @ D_sqrt
    elif normalize == 'rw':
        L = np.diag(np.clip(np.sum(adjacency, axis=1), 1, None)**(-1)) @ cm_L
    else:
        L = cm_L

    return L

def spectral_decomposition(laplacian: np.ndarray):
    """ 
    Given a Laplacian matrix computes the corresponding eigenvalues and eigenvectors

    Inputs
      laplacian [numpy.ndarray]: Laplacian matrix
    Outputs:
      lamb [numpy.ndarray]: Eigenvalues of the Laplacian
      U [numpy.ndarray]: Corresponding eigenvectors.
    """
    if np.all(laplacian == laplacian.T):
        return np.linalg.eigh(laplacian)
    else:
        return np.linalg.eig(laplacian) 

def GFT(signal: np.array, U: np.ndarray):
    """
    Function to compute the graph fourier transform 
      signal [numpy.ndarray]: float array of size n.
      U [numpy.ndarray]: matrix of size n x n containing one eigenvector per column.
    Outputs
      Graph Fourier Transform [numpy.ndarray]
    """
    return U.T @ signal

def iGFT(fourier_coefficients: np.ndarray, U: np.ndarray):
    """ 
    Inputs
      fourier_coefficients [numpy.ndarray]: float array of size n, containing a 
                                            signal represented in the spectral domain
      U [numpy.ndarray]: matrix of size n x n containing one eigenvector per column.
    Outputs
      Inverse Graph Fourier Transform [numpy.ndarray]
    """
    return U @ fourier_coefficients

## function to filter graph signal with defined filter
def filter_signal(x: np.array, spectral_response: np.array, U: np.ndarray):
    """ 
    Returns a filtered signal. The filter is defined in the spectral domain by its 
    value on each eigenvector

    Inputs
      x [numpy.ndarray]: input signal
      spectral response [numpy.ndarray]: value of the filter at each eigenvalue
      U (n x n matrix): eigenvectors (one per column).
    Outputs
      out [numpy.ndarray]: Filtered signal
    """
    x_gft = GFT(x, U)
    filter_gft = x_gft * spectral_response
    return iGFT(filter_gft, U)

def compute_smoothness(x: np.array, laplacian: np.ndarray):
    """ 
    Return the average smoothness of input graph signals

    Inputs 
      x [numpy.ndarray]: Input signal
      laplacian [numpy.ndarray] : Laplacian matrix
    Outputs
      Average smoothness of input graph signals
    """ 
    smoothness = x.T @ laplacian @ x
    return smoothness.diagonal()

### Eigenvalues and smoothness

**Firstly, we plot the eigenvalues and analyse smoothness of graph signals.**

To calculate the smoothness of our features, we calculate the inverse of  smoothness along each feature. Then, we calculate the average of the smoothness of all features. The process is indicated as following:

$$smoothness = \sum^{i=N_{features}}_{i=1} \frac{x_i^{T}Lx_i} {N_{features}}$$

where ${N_{features}} = 35$ (number of hand-crafted features)

**Marvel 2000**

In [ ]:
# compute symmetric normalized graph laplacian and calculate smoothness for Marvel 2000
L_sym_mar_2000 = compute_laplacian(adjacency_mar_2000, 'sym')
smooth_mar_2000 = compute_smoothness(features_mar_2000, L_sym_mar_2000)
lam_mar_2000, U_mar_2000 = spectral_decomposition(L_sym_mar_2000)
lam_mar_2000, U_mar_2000 = np.real(lam_mar_2000), np.real(U_mar_2000.real)

# sort eigenvalues
argsort = lam_mar_2000.argsort()
lam_mar_2000 = lam_mar_2000[argsort]
U_mar_2000 = U_mar_2000[:, argsort]

**Marvel 2001**

In [ ]:
# compute symmetric normalized graph laplacian and calculate smoothness for Marvel 2001
L_sym_mar_2001 = compute_laplacian(adjacency_mar_2001, 'sym')
smooth_mar_2001 = compute_smoothness(features_mar_2001, L_sym_mar_2001)
lam_mar_2001, U_mar_2001 = spectral_decomposition(L_sym_mar_2001)
lam_mar_2001, U_mar_2001 = np.real(lam_mar_2001), np.real(U_mar_2001.real)

# sort eigenvalues
argsort = lam_mar_2001.argsort()
lam_mar_2001 = lam_mar_2001[argsort]
U_mar_2001 = U_mar_2001[:, argsort]

**DC 2000**

In [ ]:
# compute symmetric normalized graph laplacian and calculate smoothness for DC 2000
L_sym_DC_2000 = compute_laplacian(adjacency_DC_2000, 'sym')
smooth_DC_2000 = compute_smoothness(features_DC_2000, L_sym_DC_2000)
lam_DC_2000, U_DC_2000 = spectral_decomposition(L_sym_DC_2000)
lam_DC_2000, U_DC_2000 = np.real(lam_DC_2000), np.real(U_DC_2000.real)

# sort eigenvalues
argsort = lam_DC_2000.argsort()
lam_DC_2000 = lam_DC_2000[argsort]
U_DC_2000 = U_DC_2000[:, argsort]

**DC 2001**

In [ ]:
# compute symmetric normalized graph laplacian and calculate smoothness for DC 2001
L_sym_DC_2001 = compute_laplacian(adjacency_DC_2001, 'sym')
smooth_DC_2001 = compute_smoothness(features_DC_2001, L_sym_DC_2001)
lam_DC_2001, U_DC_2001 = spectral_decomposition(L_sym_DC_2001)
lam_DC_2001, U_DC_2001 = np.real(lam_DC_2001), np.real(U_DC_2001.real)

# sort eigenvalues
argsort = lam_DC_2001.argsort()
lam_DC_2001 = lam_DC_2001[argsort]
U_DC_2001 = U_DC_2001[:, argsort]

**Results**

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))

axes[0][0].plot(lam_mar_2000, '+-')
axes[0][0].set_title('Eigenvalues $L_{sym}$ for Marvel 2000')
axes[0][0].set_ylabel('Eigenvalues')
axes[0][0].set_xlabel('Index')

axes[0][1].plot(lam_mar_2001, '+-')
axes[0][1].set_title('Eigenvalues $L_{sym}$ for Marvel 2001')
axes[0][1].set_ylabel('Eigenvalues')
axes[0][1].set_xlabel('Index')

axes[1][0].plot(lam_DC_2000, '+-')
axes[1][0].set_title('Eigenvalues $L_{sym}$ for DC 2000')
axes[1][0].set_ylabel('Eigenvalues')
axes[1][0].set_xlabel('Index')

axes[1][1].plot(lam_DC_2001, '+-')
axes[1][1].set_title('Eigenvalues $L_{sym}$ for DC 2001')
axes[1][1].set_ylabel('Eigenvalues')
axes[1][1].set_xlabel('Index')

print('Average inverse of smoothness for Marvel 2000: {:.2f}'.format(smooth_mar_2000.mean()))
print('Average inverse of smoothness for Marvel 2001: {:.2f}'.format(smooth_mar_2001.mean()))
print('Average inverse of smoothness for DC 2000: {:.2f}'.format(smooth_DC_2000.mean()))
print('Average inverse of smoothness for DC 2001: {:.2f}\n\n'.format(smooth_DC_2001.mean()))

### Tikhonov filter

From above figures and printings, we can see the graph signals are not smooth. Therefore, we will use a filter to only preserve the components associated with the smallest eigenvalues. **The Tikhonov filter** can reduce the components associated with large eigenvalues, and preserve the low frequencies. We will use it to **smooth the graph signals**. 

**Marvel 2000**

In [ ]:
## define ideal Tikhonov filter
alpha = 0.99 / np.max(lam_mar_2000)
ideal_tk = 1 / (1 + alpha * lam_mar_2000)
ideal_tk = ideal_tk[:, None]

plt.plot(lam_mar_2000, ideal_tk, '+-')
plt.title('Spectral response of Tikhonov filter for Marvel 2000')
plt.xlabel('$\lambda$')
plt.ylabel('Spectral response')
plt.savefig(os.path.join(figures_path, 'spectral_response_marvel_2000.png'))

## filter graph signals
filtered_features_mar_2000 = filter_signal(features_mar_2000, ideal_tk, U_mar_2000)

filter_smooth_mar_2000 = compute_smoothness(filtered_features_mar_2000, L_sym_mar_2000)

print('Before smoothing, the average inverse of smoothness of Marvel 2000: {:.2f}'.format(smooth_mar_2000.mean()))
print('After smoothing, the average inverse of smoothness of Marvel 2000: {:.2f}\n\n'.format(filter_smooth_mar_2000.mean()))

**Marvel 2001**

In [ ]:
## define ideal Tikhonov filter
alpha = 0.99 / np.max(lam_mar_2001)
ideal_tk = 1 / (1 + alpha * lam_mar_2001)
ideal_tk = ideal_tk[:, None]

plt.plot(lam_mar_2001, ideal_tk, '+-')
plt.title('Spectral response of Tikhonov filter for Marvel 2001')
plt.xlabel('$\lambda$')
plt.ylabel('Spectral response')
plt.savefig(os.path.join(figures_path, 'spectral_response_marvel_2001.png'))

## filter graph signals
filtered_features_mar_2001 = filter_signal(features_mar_2001, ideal_tk, U_mar_2001)

filter_smooth_mar_2001 = compute_smoothness(filtered_features_mar_2001, L_sym_mar_2001)
print('Before smoothing, the average inverse of smoothness of Marvel 2001: {:.2f}'.format(smooth_mar_2001.mean()))
print('After smoothing, the average inverse of smoothness of Marvel 2001: {:.2f}\n\n'.format(filter_smooth_mar_2001.mean()))

**DC 2000**

In [ ]:
## define ideal Tikhonov filter
alpha = 0.99 / np.max(lam_DC_2000)
ideal_tk = 1 / (1 + alpha * lam_DC_2000)
ideal_tk = ideal_tk[:, None]

plt.plot(lam_DC_2000, ideal_tk, '+-')
plt.title('Spectral response of Tikhonov filter for DC 2000')
plt.xlabel('$\lambda$')
plt.ylabel('Spectral response')
plt.savefig(os.path.join(figures_path, 'spectral_response_DC_2000.png'))

## filter graph signals
filtered_features_DC_2000 = filter_signal(features_DC_2000, ideal_tk, U_DC_2000)

filter_smooth_DC_2000 = compute_smoothness(filtered_features_DC_2000, L_sym_DC_2000)
print('Before smoothing, the average inverse of smoothness of DC 2000: {:.2f}'.format(smooth_DC_2000.mean()))
print('After smoothing, the average inverse of smoothness of DC 2000: {:.2f}\n\n'.format(filter_smooth_DC_2000.mean()))

**DC 2001**

In [ ]:
## define ideal Tikhonov filter
alpha = 0.99 / np.max(lam_DC_2001)
ideal_tk = 1 / (1 + alpha * lam_DC_2001)
ideal_tk = ideal_tk[:, None]

plt.plot(lam_DC_2001, ideal_tk, '+-')
plt.title('Spectral response of Tikhonov filter for DC 2001')
plt.xlabel('$\lambda$')
plt.ylabel('Spectral response')
plt.savefig(os.path.join(figures_path, 'spectral_response_DC_2001.png'))

## filter graph signals
filtered_features_DC_2001 = filter_signal(features_DC_2001, ideal_tk, U_DC_2001)

filter_smooth_DC_2001 = compute_smoothness(filtered_features_DC_2001, L_sym_DC_2001)
print('Before smoothing, the average inverse of smoothness of DC 2001: {:.2f}'.format(smooth_DC_2001.mean()))
print('After smoothing, the average inverse of smoothness of DC 2001: {:.2f}\n\n'.format(filter_smooth_DC_2001.mean()))

## Clustering (K-means)

As a first step, we try unsupervised methods to see whether clustering algorithms can determine a character is good, neutral or bad.

In [ ]:
def kmeansCluster(features, targets):
    """
    Performs Kmeans clustering on the provided dataset. Then prints the resulting
    accuracy and then visualises the dataset firtly with original labels and then
    with the predicted labels
    Inputs
      features [numpy.ndarray]: NxD Train matrix where N is the number of nodes and 
                                D the number of features
      labels [numpy.ndarray]: Nx1 Label vect where each node is labeld with 0,1 or 2
                              depending wether they are good, bad or neutral.
    """
    y_pred = KMeans(n_clusters=3, random_state=0).fit_predict(features)
    print('Test accuracy: {}'.format((y_pred == targets).mean()))

    print('True lables')
    visualize_features(features, targets)

    print('K-means results')
    visualize_features(features, y_pred)

In [ ]:
kmeansCluster(features_mar_2000, targets_mar_2000)

In [ ]:
kmeansCluster(features_mar_2001, targets_mar_2001)

In [ ]:
kmeansCluster(features_DC_2000, targets_DC_2000)

In [ ]:
kmeansCluster(features_DC_2001, targets_DC_2001)

Since there are only 3 classes we expect in the worst case scenario a 30% classification accuracy. The results of this method are too close to that and sometimes even worse. This can also be seen by comparing the plots with the original labels and the predicted labels. Therefore we can conclude that this method is not good enough for our goals.

## Node classification - Classical ML

In this section we will perform the following methods in order to try to classify each node/character as good, bad or neutral :

- Linear SVM (w/wout Tikhonov regularization) 
- SVM with RBF kernel (w/wout Tikhonov regularization) 
- Logistic regression (w/wout Tikhonov regularization) 

### Classifier

In order to do these we create the following classifier function responsible to perform cross validation or train the a model using the provided classifier type.

In [ ]:
from stellargraph.core.validation import separated
from numpy.core.numeric import cross
def train_classifier(features, targets, train_mask, test_mask, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='SVC'):
  """
  Inputs:
    features [numpy.ndarray]: NxD Train matrix where N is the number of nodes and 
                            D the number of features
    targets [numpy.ndarray]: Nx1 Label vect where each node is labeld with 0,1 or 2
                            depending wether they are good, bad or neutral.

    train_mask [numpy.ndarray]: NxD Train mask filled with True/False indicating
                                which elements are to be used for training or not
    test_mask [numpy.ndarray]: Nx1 Test mask filled with True/False indicating
                               which elements are to be used for testing or not
    feature_selection [bool]: Flag for weather to use feature selction, default is 
                              False. If an integer number is given than it is used as 
                              the number of highest ranked features to use
    regularization [float]: Regularization parameter
    cross__validation [bool]: If True the classifier performs a cv with scope to
                              return the best regularization parameter. If False
                              the classifier trains the whole dataset normally.
    classifier_type [str]: Classifier to use. Possible classifiers : 
                            Linear SVM          -> "LinearSVC"
                            RBF kernel SVM      -> "SVC"
                            Logistic regression -> "Logistic"

  Outputs:
     confmat: confusion matrix
     scores: feature ranking scores
  """

  seed = 0
  # split the data into training and testing sets
  X_train, X_test, y_train, y_test = features[train_mask, :], features[test_mask, :], targets[train_mask], targets[test_mask]

  # build and train the ML model
  if feature_selection:
    selector = SelectKBest(f_classif, k=feature_selection)
    X_train = selector.fit_transform(X_train, y_train)
    features = selector.get_support(indices=True)
    scores = selector.scores_
    X_test = X_test[:,features]
  
  if classifier_type == 'SVC':
      model = SVC(C=regularization, random_state=seed, tol=1e-5)
  elif classifier_type == 'LinearSVC':
      model = LinearSVC(C=regularization, random_state=seed, tol=1e-5, max_iter=2000)
  elif classifier_type == 'Logistic':
      model = LogisticRegression(C=regularization, solver='liblinear', random_state=seed, tol=1e-5)
  elif classifier_type == 'rf':
      model = RandomForestClassifier(n_estimators=100, max_depth=20, criterion='gini', class_weight = 'balanced', oob_score = True, random_state = seed)

  
  if cross_validation:
      params = {'C':[0.5, 1.0, 5.0, 10.0, 15.0]}
      kf = StratifiedKFold(n_splits=5, shuffle=True, random_state = seed) #set random_state to be 0
      clf = GridSearchCV(model, params, scoring='f1_macro', n_jobs=-1, cv=kf)
      clf.fit(X_train, y_train)
      print('Best hyperparameter:')
      print(clf.best_params_)
  else:
      print('The result of {}'.format(classifier_type))
      model.fit(X_train, y_train)

      # use the model to predict the labels of the test data
      pre_test = model.predict(X_test)
      confmat = confusion_matrix(y_test, pre_test, normalize='true')
      print('Test accuracy: {:.2f}'.format(np.mean(pre_test==y_test)))
      print('F1 score: {:.2f}'.format(f1_score(y_test, pre_test, average='macro')))

      # Display the confusion matrix
      plotConfusion(confmat)
      if feature_selection:
        return confmat, scores
      else:
        return confmat

def plotConfusion(confmat):
    """
    Plots the confusion matrix based on the predictions of our model
    """
    classes = ['Bad', 'Neutral', 'Good']
    df_cm = pd.DataFrame(confmat, index = classes, columns = classes)

    plt.figure(figsize = (12,7))
    sns.heatmap(df_cm, annot=True, cmap='Blues')
    plt.title('Confusion Matrix')

Now we have everything we need to go on and perform classical machine learning classification methods on our datasets.

### Marvel 2000 - Classic ML

**Linear SVM**

Cross validation

In [ ]:
_ = train_classifier(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=False, classifier_type='LinearSVC')

**Linear SVM with Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(filtered_features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
_ = train_classifier(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=15.0, cross_validation=False, classifier_type='SVC')

**SVM with RBF kernel and graph Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(filtered_features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=15.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
_ = train_classifier(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=False, classifier_type='Logistic')

**Logistic regression with Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(filtered_features_mar_2000, targets_mar_2000, train_mask_mar_2000, test_mask_mar_2000, regularization=1.0, cross_validation=False, classifier_type='Logistic')

### Marvel 2001 - Classic ML

**Linear SVM**

Cross validation

In [ ]:
train_classifier(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=1.0, cross_validation=False, classifier_type='LinearSVC')

**Linear SVM with Tikhonov regularization**

In [ ]:
train_classifier(filtered_features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(filtered_features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=5.0, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
train_classifier(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=5.0, cross_validation=False, classifier_type='SVC')

**SVM with RBF kernel and Tikhonov regularization**

In [ ]:
train_classifier(filtered_features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(filtered_features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=5.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
train_classifier(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=False, classifier_type='Logistic')

**Logistic regression with Tikhonov regularization**

In [ ]:
train_classifier(filtered_features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(filtered_features_mar_2001, targets_mar_2001, train_mask_mar_2001, test_mask_mar_2001, regularization=1.0, cross_validation=False, classifier_type='Logistic')

### DC 2000 - Classic ML

**Linear SVM**

Cross validation

In [ ]:
_ = train_classifier(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=False, classifier_type='LinearSVC')

**Linear SVM with Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(filtered_features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
_ = train_classifier(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=False, classifier_type='SVC')

**SVM with RBF kernel and Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(filtered_features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=15.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
_ = train_classifier(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=5.0, cross_validation=False, classifier_type='Logistic')

**Logistic regression with Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(filtered_features_DC_2000, targets_DC_2000, train_mask_DC_2000, test_mask_DC_2000, regularization=10.0, cross_validation=False, classifier_type='Logistic')

### DC 2001 - Classic ML

**Linear SVM**

Cross validation

In [ ]:
_ = train_classifier(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=1.0, cross_validation=False, classifier_type='LinearSVC')

**Linear SVM with Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = train_classifier(filtered_features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
_ = train_classifier(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=5.0, cross_validation=False, classifier_type='SVC')

**SVM with RBF kernel and Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = train_classifier(filtered_features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=10.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
_ = train_classifier(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=5.0, cross_validation=False, classifier_type='Logistic')

**Logistic regression with Tikhonov regularization**

In [ ]:
_ = train_classifier(filtered_features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=0.5, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = train_classifier(filtered_features_DC_2001, targets_DC_2001, train_mask_DC_2001, test_mask_DC_2001, regularization=5.0, cross_validation=False, classifier_type='Logistic')

## Node embedding (GNN, GTA)

In order to improve our results we will now use Graph Neural Networks instead of Classical Machine Learning to classify our characters. In this section we will define the building blocks, models and all revelant functions that we will later use to train and evaluate our results.

In [ ]:
# Check for a GPU -  Graphical Processing Unit
if torch.cuda.is_available():
    device = 'cuda'
else:
    print("No GPU :(")
    device = 'cpu'

### Building Blocks


GNN Blocks :

In [ ]:
class GNNBlock1(nn.Module):
    def __init__(self, nb_features: int, embedding_dim: int, drop_out: float) -> None:
        super().__init__()
        self.conv1 = pyg.nn.GraphConv(nb_features, embedding_dim)
        
        if drop_out:
            self.dropout = nn.Dropout(p=drop_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        if self.dropout:
            x = self.dropout(x)
        return x

In [ ]:
class GNNBlock2(nn.Module):
    def __init__(self, nb_features: int, embedding_dim: int, drop_out: float) -> None:
        super().__init__()
        self.conv1 = pyg.nn.GATConv(nb_features, embedding_dim)
        
        if drop_out:
            self.dropout = nn.Dropout(p=drop_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        if self.dropout:
            x = self.dropout(x)
        return x

In [ ]:
class GNNBlock3(nn.Module):
    def __init__(self, nb_features: int, embedding_dim: int, drop_out: float) -> None:
        super().__init__()
        self.conv1 = pyg.nn.ChebConv(nb_features, embedding_dim, K=7)
        
        if drop_out:
            self.dropout = nn.Dropout(p=drop_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        if self.dropout:
            x = self.dropout(x)
        return x

In [ ]:
class GNNBlock4(nn.Module):
    def __init__(self, nb_features: int, embedding_dim: int, drop_out: float) -> None:
        super().__init__()
        self.linear = nn.Linear(nb_features, embedding_dim)
        self.conv1 = pyg.nn.GINConv(self.linear)
        
        if drop_out:
            self.dropout = nn.Dropout(p=drop_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        if self.dropout:
            x = self.dropout(x)
        return x

Model Architecture :

In [ ]:
class GNN(nn.Module):
    """
    Puts the same building block 5 times in series, each time increasing the resolution
    """
    def __init__(self, GNNBlock, in_features: int, embedding_dim: int, hidden_dims=[32, 64, 128, 256], drop_pro=0.3) -> None:
        super().__init__()
        layers = []
        layers.append(GNNBlock(in_features, hidden_dims[0], drop_pro))
        for i, hidden_dim in enumerate(hidden_dims[1:]):
            layers.append(GNNBlock(hidden_dims[i], hidden_dim, drop_pro))
        layers.append(GNNBlock(hidden_dims[-1], embedding_dim, drop_pro))
        self.gnn = nn.ModuleList(layers)      

    def forward(self, x, edge_index):
        for layer in self.gnn:
            x = layer(x, edge_index)        
        return x

In [ ]:
class NodeClassifier(nn.Module):
    """
    Applies firstly a provided GNN block and then passes the input through a fully
    connected linear neural network.
    """
    def __init__(self, gnn_block: nn.Module, embedding_dim: int, num_classes: int) -> None:
        super().__init__()
        self.gnn_block = gnn_block
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(embedding_dim, num_classes),
            nn.Dropout(p=0.1),
        )

    def forward(self, x, edge_index) -> torch.Tensor:
        x = self.gnn_block(x, edge_index)
        x = self.classifier(x)
        return x

Relevant functions :

In [ ]:
def train(

    model: nn.Module,
    data: Data,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,
    nb_epochs: int,
):
    """
    Training our model
    """
    for epoch in tqdm(range(nb_epochs), total=nb_epochs):
        ## get predicted output
        output = model(data.x, data.edge_index)
        ## calculate loss
        loss = loss_fn(output[data.train_mask,:], data.y[data.train_mask]) 
               
        ## optimizer updates
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
def evaluate(model: nn.Module, metric: torchmetrics.Metric, data: Data, mask: torch.Tensor):
    """
    Evaluates our model
    """
    model.eval()  # Deactivate dropout
    with torch.no_grad():
        pred = model(data.x, data.edge_index).softmax(dim=1)
        _ = metric(pred[mask, :], data.y[mask])        
        #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

    return metric.compute().item()

In [ ]:
def confusion_matrix_f1_score(model: nn.Module, data: Data, mask: torch.Tensor):
    """
    Plots the confusion matrix with the f1 score instead of accuracy
    """
    model.eval()
    label = data.y[mask]
    y_pred, y_true = [], label.data.cpu().numpy()

    with torch.no_grad():
        pred = model(data.x, data.edge_index).softmax(dim=1)
        _, predicted = pred.max(1)

        test_pred = predicted.data.cpu().numpy()
        y_pred = test_pred[mask.data.cpu().numpy()]
    

    classes = ['Bad', 'Neutral', 'Good']
    c_matrix = confusion_matrix(y_true, y_pred, normalize='true')
    df_cm = pd.DataFrame(c_matrix, index = classes, columns = classes)

    plt.figure(figsize = (12,7))
    print('F1 score: {}'.format(f1_score(y_true, y_pred, average='macro')))
    sns.heatmap(df_cm, annot=True, cmap='Blues')
    plt.title('Confusion Matrix')

### Example

Simply showing an example of what we did so far.

In [ ]:
##### test model
gnn = GNN(GNNBlock1, 3, 512, drop_pro=0.5)
x = torch.randn(6,3)
edge_index = torch.arange(6, dtype=torch.long).reshape(2,3)
output = gnn(x, edge_index)
assert output.shape == (6, 512)

In [ ]:
print(gnn)

### Debugging

In [ ]:
### make the empty current alias be empty string, because if we do not make the change,
### a bug ('TypeError: new(): invalid data type 'str'') will occur.
def change_empty_current_alias(graph):
    for node in graph.nodes:
        if graph.nodes[node]['Current Alias'] is '':
            graph.nodes[node]['Current Alias'] = ' '

In [ ]:
### make the empty current alias be empty string, because if we do not make the change,
### a bug ('TypeError: new(): invalid data type 'str'') will occur.
change_empty_current_alias(graph_mar_2000)
change_empty_current_alias(graph_mar_2001)
change_empty_current_alias(graph_DC_2000)
change_empty_current_alias(graph_DC_2001)


pyg_graph_mar_2000 = from_networkx(graph_mar_2000)
pyg_graph_mar_2001 = from_networkx(graph_mar_2001)
pyg_graph_DC_2000 = from_networkx(graph_DC_2000)
pyg_graph_DC_2001 = from_networkx(graph_DC_2001)

In [ ]:
pyg_graph_mar_2000

In [ ]:
pyg_graph_mar_2000.x = pyg_graph_mar_2000.x.to(torch.float32)
pyg_graph_mar_2001.x = pyg_graph_mar_2001.x.to(torch.float32)
pyg_graph_DC_2000.x = pyg_graph_DC_2000.x.to(torch.float32)
pyg_graph_DC_2001.x = pyg_graph_DC_2001.x.to(torch.float32)

Tikhonov filtered graph

In [ ]:
filtered_pyg_graph_mar_2000 = pyg_graph_mar_2000.clone()
filtered_pyg_graph_mar_2000.x = torch.from_numpy(filtered_features_mar_2000).to(torch.float32)

filtered_pyg_graph_mar_2001 = pyg_graph_mar_2001.clone()
filtered_pyg_graph_mar_2001.x = torch.from_numpy(filtered_features_mar_2001).to(torch.float32)

filtered_pyg_graph_DC_2000 = pyg_graph_DC_2000.clone()
filtered_pyg_graph_DC_2000.x = torch.from_numpy(filtered_features_DC_2000).to(torch.float32)

filtered_pyg_graph_DC_2001 = pyg_graph_DC_2001.clone()
filtered_pyg_graph_DC_2001.x = torch.from_numpy(filtered_features_DC_2001).to(torch.float32)

## Train and evaluate GNN classifier

### Marvel 2000 - GNN

In [ ]:
# embedding dimensions
EMBEDDING_DIM = 128

# learning rate
LR = 1e-2

# weight decay
WD = 1e-3

gnn_mar_2000 = GNN(GNNBlock3, pyg_graph_mar_2000.num_features, EMBEDDING_DIM, hidden_dims=[8, 16, 32, 64], drop_pro=0.1).to(device)
model = NodeClassifier(
    gnn_mar_2000,
    embedding_dim=EMBEDDING_DIM,
    num_classes=3
).to(device)

loss_fn = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

nb_epochs = 1000

data = pyg_graph_mar_2000.to(device)

train(model, data, loss_fn, optimizer, nb_epochs)

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print(model)

In [ ]:
print('Result for Marvel 2000')
accuracy_train = evaluate(model, torchmetrics.Accuracy().to(device), data, data.train_mask)
print("Train accuracy:", accuracy_train)
print("Test accuracy:", evaluate(model, torchmetrics.Accuracy().to(device), data, data.test_mask))

confusion_matrix_f1_score(model, data, data.test_mask)

In [ ]:
nodeEmbedding_mar_2000 = gnn_mar_2000(data.x, data.edge_index).data.cpu().numpy()

In [ ]:
#Targets visualisation
visualize_features(features_mar_2000, targets_mar_2000)

In [ ]:
#Predictions visualisation
visualize_features(nodeEmbedding_mar_2000, targets_mar_2000)

From above figures, we can see GNN can provide more meaningful features than hand-crafted features. It is very clear in TSNE plot.

**Using Tikhonov regularization**

In [ ]:
# embedding dimensions
EMBEDDING_DIM = 128

# learning rate
LR = 1e-2

# weight decay
WD = 1e-3

gnn_mar_2000 = GNN(GNNBlock3, filtered_pyg_graph_mar_2000.num_features, EMBEDDING_DIM, hidden_dims=[8, 16, 32, 64], drop_pro=0.1).to(device)
model = NodeClassifier(
    gnn_mar_2000,
    embedding_dim=EMBEDDING_DIM,
    num_classes=3
).to(device)

loss_fn = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

nb_epochs = 1000

data = filtered_pyg_graph_mar_2000.to(device)

train(model, data, loss_fn, optimizer, nb_epochs)

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print('Result for Marvel 2000')
accuracy_train = evaluate(model, torchmetrics.Accuracy().to(device), data, data.train_mask)
print("Train accuracy:", accuracy_train)
print("Test accuracy:", evaluate(model, torchmetrics.Accuracy().to(device), data, data.test_mask))

confusion_matrix_f1_score(model, data, data.test_mask)

### Marvel 2001 - GNN

In [ ]:
# embedding dimensions
EMBEDDING_DIM = 128

# learning rate
LR = 1e-2

# weight decay
WD = 1e-3

gnn_mar_2001 = GNN(GNNBlock3, pyg_graph_mar_2001.num_features, EMBEDDING_DIM, hidden_dims=[8, 16, 32, 64], drop_pro=0.1).to(device)
model = NodeClassifier(
    gnn_mar_2001,
    embedding_dim=EMBEDDING_DIM,
    num_classes=3
).to(device)

loss_fn = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

nb_epochs = 1000

data = pyg_graph_mar_2001.to(device)

train(model, data, loss_fn, optimizer, nb_epochs)

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print('Result for Marvel 2001')
accuracy_train = evaluate(model, torchmetrics.Accuracy().to(device), data, data.train_mask)
print("Train accuracy:", accuracy_train)
print("Test accuracy:", evaluate(model, torchmetrics.Accuracy().to(device), data, data.test_mask))

confusion_matrix_f1_score(model, data, data.test_mask)

In [ ]:
nodeEmbedding_mar_2001 = gnn_mar_2001(data.x, data.edge_index).data.cpu().numpy()

In [ ]:
# Targets Visualisation
visualize_features(features_mar_2001, targets_mar_2001)

In [ ]:
# Predictions visualisation
visualize_features(nodeEmbedding_mar_2001, targets_mar_2001)

From above figures, we can see GNN can provide more meaningful features than hand-crafted features. It is very clear in TSNE plot.

### DC 2000 - GNN

In [ ]:
# embedding dimensions
EMBEDDING_DIM = 128

# learning rate
LR = 1e-2

# weight decay
WD = 1e-3

gnn_dc_2000 = GNN(GNNBlock3, pyg_graph_DC_2000.num_features, EMBEDDING_DIM, hidden_dims=[8, 16, 32, 64], drop_pro=0.1).to(device)
model = NodeClassifier(
    gnn_dc_2000,
    embedding_dim=EMBEDDING_DIM,
    num_classes=3
).to(device)

loss_fn = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

nb_epochs = 1000

data = pyg_graph_DC_2000.to(device)

train(model, data, loss_fn, optimizer, nb_epochs)

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print('Result for DC 2000')
accuracy_train = evaluate(model, torchmetrics.Accuracy().to(device), data, data.train_mask)
print("Train accuracy:", accuracy_train)
print("Test accuracy:", evaluate(model, torchmetrics.Accuracy().to(device), data, data.test_mask))

confusion_matrix_f1_score(model, data, data.test_mask)

In [ ]:
nodeEmbedding_dc_2000 = gnn_dc_2000(data.x, data.edge_index).data.cpu().numpy()

In [ ]:
# Targets visualisation
visualize_features(features_DC_2000, targets_DC_2000)

In [ ]:
# Predictions visualisation
visualize_features(nodeEmbedding_dc_2000, targets_DC_2000)

From above figures, we can see GNN can provide more meaningful features than hand-crafted features. It is very clear in TSNE plot.

### DC 2001 - GNN

In [ ]:
# embedding dimensions
EMBEDDING_DIM = 128

# learning rate
LR = 1e-2

# weight decay
WD = 1e-3

gnn_dc_2001 = GNN(GNNBlock3, pyg_graph_DC_2001.num_features, EMBEDDING_DIM, hidden_dims=[8, 16, 32, 64], drop_pro=0.1).to(device)
model = NodeClassifier(
    gnn_dc_2001,
    embedding_dim=EMBEDDING_DIM,
    num_classes=3
).to(device)

loss_fn = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

nb_epochs = 1000

data = pyg_graph_DC_2001.to(device)

train(model, data, loss_fn, optimizer, nb_epochs)

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print('Result for DC 2001')
accuracy_train = evaluate(model, torchmetrics.Accuracy().to(device), data, data.train_mask)
print("Train accuracy:", accuracy_train)
print("Test accuracy:", evaluate(model, torchmetrics.Accuracy().to(device), data, data.test_mask))

confusion_matrix_f1_score(model, data, data.test_mask)

In [ ]:
nodeEmbedding_dc_2001 = gnn_dc_2001(data.x, data.edge_index).data.cpu().numpy()

In [ ]:
# Targets visualisation
visualize_features(features_DC_2001, targets_DC_2001)

In [ ]:
#Predictions visualisation
visualize_features(nodeEmbedding_dc_2001, targets_DC_2001)

From above figures, we can see GNN can provide more meaningful features than hand-crafted features. It is very clear in TSNE plot.

## Affiliation classification

**The pipeline of the affiliation classification is demonstrated as following:**



1. Assign labels for different affiliations (calculate the mean value of node labels) 
2. Pooling the node embeddings of each affiliation
3. Train SVM classifier to do the classification 



### Assign labels

In [ ]:
def transformLabel(x):
    """
    Transforms mean value of labels of nodes in the following way
      x in [-1,-0.33)    -> x=-1
      x in [-0.33, 0.34) -> x=0
      x in [0.34, 1]     -> x=1 

    Inputs 
      x [float]: Mean value of feature "Good" for characters in specific
                 affiliation
    Outputs
      Transformation of x as explained above
    """
    if -1 <= x < -0.33:
        return -1
    elif -0.33 <= x < 0.34:
        return 0
    elif 0.34<= x <= 1:
        return 1

In [ ]:
def assignLabels(df):
    """
    Assigns a Good (1), Bad (-1) or Neutral (0) to each affiliation based on the 
    mean value of "Good" for the characters affiliated with it

    Inputs
      df [pandas.DataFrame]: Initial dataframe (parquet provided)
    Outputs
      df_affiliations [pandas.DataFrame] : Dataframe of the following form
         
                     |Affiliation|Label|Id
    """
    df_nodes = df.explode('Affiliation')[['Id', 'Affiliation']]
    df_affiliation = df.explode('Affiliation').groupby('Affiliation').mean().reset_index()[['Affiliation', 'Good']]
    df_affiliation.rename(columns = {'Good': 'Label'}, inplace = True)
    df_affiliation['Label'] = df_affiliation['Label'].apply(transformLabel)

    df_affiliation = df_affiliation.merge(df_nodes, on='Affiliation')
    return df_affiliation

In [ ]:
def checkTopAffLabels(df):
    """
    Returns top 10 most popular affiliations and their label

    Inputs
      df [pandas.DataFrame]: Dataframe with the Affiliations labeled
    """
    top_aff = df['Affiliation'].value_counts()[:10].index
    for i, aff in enumerate(top_aff):
        print('Top {} popular Affilation: {}, Label: {}'.format(i+1, aff, df[df['Affiliation'] == aff].Label.values[0]))

In [ ]:
# Affiliations labeling for Marvel 2000
affiliation_mar_2000 = assignLabels(marvel_data_2000)
# Affiliations labeling for Marvel 2001
affiliation_mar_2001 = assignLabels(marvel_data_2001)
# Affiliations labeling for DC 2000
affiliation_DC_2000 = assignLabels(DC_data_2000)
# Affiliations labeling for DC 2001
affiliation_DC_2001 = assignLabels(DC_data_2001)

Showing an example of the created dataframes : 

In [ ]:
affiliation_mar_2000

To check our assigned lables are meaningful, we verify the lables of the popular affiliations.

In [ ]:
checkTopAffLabels(affiliation_mar_2000)

In [ ]:
checkTopAffLabels(affiliation_mar_2001)

**From above printing, we can see the our assigned labels are meaningful, because X-men and Avergers are indeed good affiliation (which everybody knows). Also, Masters of Evil is evil affiliation (which every body knows).**

In [ ]:
checkTopAffLabels(affiliation_DC_2000)

In [ ]:
checkTopAffLabels(affiliation_DC_2001)

**From above printing, we can see the our assigned labels are meaningful, because Black Lantern Corps and All-Star Squadron are indeed good affiliation (which everybody knows). Also, Secret Society of Super-Villains III is evil affiliation (which every body knows).**

**Label distribution**

In [ ]:
def plotAffLabelDistribution(df, axIndex, name='Marvel 2000'):
    """
    Plots a barplot with the number of good, bad and neutral affiliations
    in our dataset

    Inputs 
      df [pandas.DataFrame] : Dataset dataframe
      axIndex [matplotlib.axes._subplots.AxesSubplot] : Position of the graph
      name [str] : Name of the graph to plot. Can be one of the following 
                   ["Marvel 2000", "Marvel 2001", "DC 2000", "DC 2001"]
                   (just to give a title)
    """
    labels = df['Label'].value_counts()
    labels = labels.rename(index={1: 'Good', 0:'Neutral', -1:'Bad'})
    ax = sns.barplot(x=labels.index, y=labels.values, ax=axIndex)
    ax.set_title('{} Affiliation labels distribution'.format(name))
    ax.set_ylabel('Number')
    ax.set_xlabel('Class')

In [ ]:
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
plotAffLabelDistribution(affiliation_mar_2000, axes[0][0], name='Marvel 2000')
plotAffLabelDistribution(affiliation_mar_2001, axes[0][1], name='Marvel 2001')
plotAffLabelDistribution(affiliation_DC_2000, axes[1][0], name='DC 2000')
plotAffLabelDistribution(affiliation_DC_2001, axes[1][1], name='DC 2001')

From above figures, we can see the labels are unbalanced. Therefore, we will split the data into train set and test set in stratified fashion. We will also report not only the test accuracy but also the **F1 score and confusion matrix**. 

### Affiliation Embedding (Average pooling)

In [ ]:
def avgPooling(nodeEmbedding, df):
    affiliation_list = df.Affiliation.unique()
    aff_embedding = np.zeros((len(affiliation_list), nodeEmbedding.shape[1]))
    labels = np.zeros(len(affiliation_list))

    for i, aff in enumerate(affiliation_list):
        mask = df[df['Affiliation'] == aff]['Id'].values
        aff_embedding[i] = nodeEmbedding[mask, :].mean(axis=0)
        labels[i] = int(df[df['Affiliation'] == aff]['Label'].values[0]) + 1
    
    return aff_embedding, labels, affiliation_list

**Marvel 2000 Affiliations feature dataset**

In [ ]:
aff_embedding_mar_2000, aff_labels_mar_2000, aff_mar_2000 = avgPooling(nodeEmbedding_mar_2000, affiliation_mar_2000)

In [ ]:
visualize_features(aff_embedding_mar_2000, aff_labels_mar_2000)

**Marvel 2001 Affiliations feature dataset**

In [ ]:
aff_embedding_mar_2001, aff_labels_mar_2001, aff_mar_2001 = avgPooling(nodeEmbedding_mar_2001, affiliation_mar_2001)

In [ ]:
visualize_features(aff_embedding_mar_2001, aff_labels_mar_2001)

**DC 2000 Affiliations feature dataset**

In [ ]:
aff_embedding_DC_2000, aff_labels_DC_2000, aff_DC_2000 = avgPooling(nodeEmbedding_dc_2000, affiliation_DC_2000)

In [ ]:
visualize_features(aff_embedding_DC_2000, aff_labels_DC_2000)

**DC 2001 Affiliations feature dataset**

In [ ]:
aff_embedding_DC_2001, aff_labels_DC_2001, aff_DC_2001 = avgPooling(nodeEmbedding_dc_2001, affiliation_DC_2001)

In [ ]:
visualize_features(aff_embedding_DC_2001, aff_labels_DC_2001)

### Affiliation Classical ML Classification

In [ ]:
def affiliation_classifier(features, targets, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='SVC'):
  """
  Inputs:
    features [numpy.ndarray]: NxD Train matrix where N is the number of nodes/affiliations 
                              and D the number of features
    targets [numpy.ndarray]: Nx1 Label vect where each node is labeld with 0,1 or 2
                            depending wether they are good, bad or neutral.
    train_mask [numpy.ndarray]: NxD Train mask filled with True/False indicating
                                which elements are to be used for training or not
    test_mask [numpy.ndarray]: Nx1 Test mask filled with True/False indicating
                               which elements are to be used for testing or not
    feature_selection [bool]: Flag for weather to use feature selction, default is 
                              False. If an integer number is given than it is used as 
                              the number of highest ranked features to use
    regularization [float]: Regularization parameter
    cross__validation [bool]: If True the classifier performs a cv with scope to
                              return the best regularization parameter. If False
                              the classifier trains the whole dataset normally.
    classifier_type [str]: Classifier to use. Possible classifiers : 
                            Linear SVM          -> "LinearSVC"
                            RBF kernel SVM      -> "SVC"
                            Logistic regression -> "Logistic"

  Outputs:
     confmat: confusion matrix
     scores: feature ranking scores
  """
  seed = 0 

  # split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(features, targets, stratify=targets, test_size=0.25, random_state = seed)

  # build and train the ML model
  scaler = StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  if feature_selection:
    selector = SelectKBest(f_classif, k=feature_selection)
    X_train = selector.fit_transform(X_train, y_train)
    features = selector.get_support(indices=True)
    scores = selector.scores_
    X_test = X_test[:,features]
  
  if classifier_type == 'SVC':
      model = SVC(C=regularization, random_state=seed, tol=1e-5)
  elif classifier_type == 'LinearSVC':
      model = LinearSVC(random_state=seed, tol=1e-2, max_iter=1000)
  elif classifier_type == 'Logistic':
      model = LogisticRegression(random_state=seed, tol=1e-2, max_iter=1000)

  model.fit(X_train, y_train)


  if cross_validation:
      params = {'C':[0.5, 1.0, 5.0, 10.0, 15.0]}
      kf = StratifiedKFold(n_splits=5, shuffle=True, random_state = seed) #set random_state to be 0
      clf = GridSearchCV(model, params, scoring='f1_macro', n_jobs=-1, cv=kf)
      clf.fit(X_train, y_train)
      print('Best hyperparameter:')
      print(clf.best_params_)
  else:
      print('The result of {}'.format(classifier_type))
      model.fit(X_train, y_train)

      # use the model to predict the labels of the test data
      pre_test = model.predict(X_test)
      confmat = confusion_matrix(y_test, pre_test, normalize='true')
      print('Test accuracy: {}'.format(np.mean(pre_test==y_test)))
      print('F1 score: {}'.format(f1_score(y_test, pre_test, average='macro')))

      # Display the confusion matrix
      plotConfusion(confmat)
      if feature_selection:
        return confmat, scores
      else:
        return confmat

### Marvel 2000

**Linear SVM**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_mar_2000, aff_labels_mar_2000, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_mar_2000, aff_labels_mar_2000, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_mar_2000, aff_labels_mar_2000, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_mar_2000, aff_labels_mar_2000, feature_selection = False, regularization=10.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_mar_2000, aff_labels_mar_2000, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = affiliation_classifier(aff_embedding_mar_2000, aff_labels_mar_2000, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='Logistic')

### Marvel 2001

**Linear SVM**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_mar_2001, aff_labels_mar_2001, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_mar_2001, aff_labels_mar_2001, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_mar_2001, aff_labels_mar_2001, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_mar_2001, aff_labels_mar_2001, feature_selection = False, regularization=10.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_mar_2001, aff_labels_mar_2001, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = affiliation_classifier(aff_embedding_mar_2001, aff_labels_mar_2001, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='Logistic')

### DC 2000

**Linear SVM**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_DC_2000, aff_labels_DC_2000, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_DC_2000, aff_labels_DC_2000, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_DC_2000, aff_labels_DC_2000, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_DC_2000, aff_labels_DC_2000, feature_selection = False, regularization=15.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_DC_2000, aff_labels_DC_2000, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = affiliation_classifier(aff_embedding_DC_2000, aff_labels_DC_2000, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='Logistic')

### DC 2001

**Linear SVM**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_DC_2001, aff_labels_DC_2001, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='LinearSVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_DC_2001, aff_labels_DC_2001, feature_selection = False, regularization=0.5, cross_validation=False, classifier_type='LinearSVC')

**SVM with RBF kernel**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_DC_2001, aff_labels_DC_2001, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='SVC')

In [ ]:
_ = affiliation_classifier(aff_embedding_DC_2001, aff_labels_DC_2001, feature_selection = False, regularization=5.0, cross_validation=False, classifier_type='SVC')

**Logistic regression**

Cross validation

In [ ]:
affiliation_classifier(aff_embedding_DC_2001, aff_labels_DC_2001, feature_selection = False, regularization=10.0, cross_validation=True, classifier_type='Logistic')

In [ ]:
_ = affiliation_classifier(aff_embedding_DC_2001, aff_labels_DC_2001, feature_selection = False, regularization=5.0, cross_validation=False, classifier_type='Logistic')